In [0]:
%sql
--To check if there are duplicates in table contactos before the join , just for security

SELECT
  id,
  data_criacao_da_lead,
  COUNT(*) AS cnt
FROM sc_gold.contactos_pbs
GROUP BY id, data_criacao_da_lead
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, data_criacao_da_lead;


In [0]:
%sql
DESCRIBE sc_gold.`leads_pbs`;
-- or
SHOW COLUMNS IN sc_gold.`leads_pbs`;


In [0]:
%sql
SELECT
  created_time AS lead_created_time
FROM sc_silver.leads_pbs
LIMIT 50;


In [0]:
%sql
SELECT
  data_criacao_da_lead
FROM sc_silver.contactos_pbs
LIMIT 50;

In [0]:

%sql

SELECT * FROM sc_gold.campaigns             LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.leads_pbs             LIMIT 50;


In [0]:
%sql

SELECT * FROM sc_gold.contactos_pbs         LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.propostas_realizadas  LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.deals                 LIMIT 50;

In [0]:
%sql
SELECT * FROM sc_gold.contactos_pbs         LIMIT 50;

In [0]:
# Build and run: CREATE OR REPLACE TABLE sc_gold.contactos_pbs_2 AS SELECT ... FROM sc_gold.contactos_pbs

# 1) Read source columns in order
cols = spark.sql("""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'contactos_pbs'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns you do NOT want renamed (e.g., keep id as-is)
skip = set()   # e.g., {'id'}

# 2) Build the select list with aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_contactos') or c in skip) else f"{c}_contactos"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build the full CREATE TABLE AS SELECT statement
create_sql = f"""
CREATE OR REPLACE TABLE sc_gold.contactos_pbs_2 AS
SELECT {select_sql}
FROM sc_gold.contactos_pbs
"""

print(create_sql)  # shows exactly what will run
spark.sql(create_sql)

# 4) Quick verification
spark.sql("DESCRIBE TABLE sc_gold.contactos_pbs_2").show(truncate=False)
spark.sql("SELECT * FROM sc_gold.contactos_pbs_2 LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.contactos_pbs_2

In [0]:
# Source and target
source_table = "sc_gold.propostas_realizadas"
target_table = "sc_gold.propostas_realizadas2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'propostas_realizadas'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged
skip = set()   # e.g., {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_propostas') or c in skip) else f"{c}_propostas"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Show what will run
spark.sql(create_sql)

# 4) Verify
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.propostas_realizadas2

In [0]:
# Source and target
source_table = "sc_gold.deals"
target_table = "sc_gold.deals2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'deals'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged (e.g., primary keys)
skip = set()   # Example: {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_deals') or c in skip) else f"{c}_deals"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Show generated SQL
spark.sql(create_sql)

# 4) Verify structure and sample data
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.deals2

In [0]:
# Source and target
source_table = "sc_gold.campaigns"
target_table = "sc_gold.campaigns2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'campaigns'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged (like primary keys)
skip = set()   # Example: {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_campaigns') or c in skip) else f"{c}_campaigns"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Preview generated SQL
spark.sql(create_sql)

# 4) Verify new table
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.campaigns2

In [0]:
%sql
SELECT COUNT(*) AS strict_matches
FROM sc_gold.leads_pbs l
JOIN sc_gold.contactos_pbs_2 c
  ON  l.converted_contact = c.id_contactos
  AND l.created_time      = c.data_criacao_da_lead_contactos;


In [0]:
%sql
WITH joined AS (
  SELECT
    l.*,
    c.*
  FROM sc_gold.leads_pbs       AS l
  JOIN sc_gold.contactos_pbs_2 AS c
    ON  l.converted_contact = c.id_contactos
   AND l.created_time      = c.data_criacao_da_lead_contactos
)
SELECT COUNT(*) AS check_count FROM joined;   -- should be ~118329


In [0]:
%sql
-- (optional) ensure you're in the right place
-- USE CATALOG workspace;
-- USE SCHEMA sc_gold;

CREATE OR REPLACE TABLE sc_gold.Features_Table
USING DELTA
AS
SELECT
  l.*,
  c.*
FROM sc_gold.leads_pbs       AS l
JOIN sc_gold.contactos_pbs_2 AS c
  ON  l.converted_contact = c.id_contactos
  AND l.created_time      = c.data_criacao_da_lead_contactos;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  f.*,
  p.*   -- assumes Propostas columns are already suffixed (e.g., *_propostas)
FROM sc_gold.Features_Table AS f
LEFT JOIN sc_gold.propostas_realizadas2 AS p
  ON f.id_contactos = p.id_contacto_propostas;   -- keep all Feature rows


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  f.*,
  d.*   -- assumes Deals columns are already suffixed (e.g., *_deals)
FROM sc_gold.Features_Table AS f
LEFT JOIN sc_gold.deals2 AS d
  ON f.id_contacto_propostas = d.id_contacto_deals;   -- keep all Feature rows


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  f.*,
  cp.*   -- assumes campaigns columns are already suffixed (e.g., *_campaigns)
FROM sc_gold.Features_Table AS f
LEFT JOIN sc_gold.campaigns2 AS cp
  ON f.campanha_deals = cp.id_campaigns;   -- join via Features_Table.campanha_deals -> campaigns.id_campaigns


In [0]:
%%sql
CREATE OR REPLACE TABLE sc_gold.fact_table AS
SELECT
    l.id,
  l.converted_contact,
  l.created_time,
  c.id_contactos,
  c.data_criacao_da_lead_contactos
FROM sc_gold.leads_pbs AS l
JOIN sc_gold.contactos_pbs_2 AS c
  ON l.converted_contact = c.id_contactos
 AND l.created_time = c.data_criacao_da_lead_contactos;


In [0]:
%%sql
CREATE OR REPLACE TABLE sc_gold.fact_table AS
SELECT
  f.*,
 
    p.id_contacto_propostas,
  p.id_proposta_realizada_propostas

FROM sc_gold.fact_table AS f
LEFT JOIN sc_gold.propostas_realizadas2 AS p
  ON p.id_contacto_propostas = f.id_contactos;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Fact_table AS
SELECT
  f.*,
  d.id_deals,
  d.id_contacto_deals,
  d.campanha_deals
FROM sc_gold.Fact_table AS f
LEFT JOIN sc_gold.deals2 AS d
  ON f.id_contacto_propostas = d.id_contacto_deals;


In [0]:
# snapshot the current table to a temp view
spark.sql("CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM sc_gold.fact_table")

to_drop = {'converted_contact','data_criacao_da_lead_contactos','id_contacto_propostas','id_contacto_deals'}
cols = [f"`{c}`" for c in spark.table("sc_gold.fact_table").columns if c not in to_drop]

spark.sql(f"""
CREATE OR REPLACE TABLE sc_gold.fact_table
USING DELTA AS
SELECT {', '.join(cols)} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
-- 1) Summary counts of duplicates on (id, created_time)
WITH d AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_gold.leads_pbs
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)                    AS duplicated_keys,          -- number of (id,created_time) combos with >1 row
  SUM(cnt)                    AS rows_in_duplicated_keys,  -- total rows participating in duplicates
  SUM(cnt) - COUNT(*)         AS extra_rows_to_dedup       -- rows to drop if you keep 1 per key
FROM d;

-- 2) See the duplicated keys and their sizes (top 100)
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_gold.leads_pbs
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id
LIMIT 100;


In [0]:
%sql
-- SUMMARY: how many duplicate (id, created_time) keys exist in sc_gold.features ( this is normal , can have multiple proposals)
WITH d AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_gold.features_table
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)            AS duplicated_keys,
  SUM(cnt)            AS rows_in_duplicated_keys,
  SUM(cnt) - COUNT(*) AS extra_rows_to_dedup
FROM d;


In [0]:
%sql
-- SUMMARY: how many duplicate (id_propostas, created_time) keys exist
WITH d AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_gold.propostas_realizadas
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)            AS duplicated_keys,          -- number of key pairs with >1 row
  SUM(cnt)            AS rows_in_duplicated_keys,  -- total rows across those keys
  SUM(cnt) - COUNT(*) AS extra_rows_to_dedup       -- rows you’d remove if keeping 1 per key
FROM d;

-- DETAILS: which keys are duplicated (top 100 by size)
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_gold.propostas_realizadas
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id
LIMIT 100;

-- OPTIONAL: list the actual duplicate rows
SELECT t.*
FROM sc_gold.propostas_realizadas t
JOIN (
  SELECT id, created_time
  FROM sc_gold.propostas_realizadas
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
) d
ON t.id = d.id AND t.created_time = d.created_time
ORDER BY t.id, t.created_time;


In [0]:
%sql
WITH d AS (
  SELECT
    `id`,
    `id_deals`,
    `id_proposta_realizada_propostas`,
    COUNT(*) AS cnt
  FROM sc_gold.features_table
  GROUP BY `id`, `id_deals`, `id_proposta_realizada_propostas`
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)              AS duplicated_keys,
  SUM(d.cnt)            AS rows_in_duplicated_keys,
  SUM(d.cnt) - COUNT(*) AS extra_rows_to_dedup
FROM d AS d;


In [0]:
%sql
WITH d AS (
  SELECT
    `id`,
    `id_deals`,
    `id_proposta_realizada_propostas`,
    COUNT(*) AS cnt
  FROM sc_gold.fact_table
  GROUP BY `id`, `id_deals`, `id_proposta_realizada_propostas`
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)              AS duplicated_keys,
  SUM(d.cnt)            AS rows_in_duplicated_keys,
  SUM(d.cnt) - COUNT(*) AS extra_rows_to_dedup
FROM d AS d;


In [0]:
%sql
SELECT * FROM sc_gold.Fact_table

In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
# 1) Build keep list to Ml Models
cols_to_delete = [  # the  names we listed
    "_fivetran_id","id","lead_owner","lead_owner_name","consentimento",
    "id_lead","chave_instalacao_bd_rede","chave_concessao_bd_rede","chave_contrato_concessionario_bd_rede",
    "chave_agrupamento_performance_vendas","chave_agrupamento_cliente",
    "data_de_validade_de_consentimento_hyundai","lead_response_time","converted_date_time",
    "_fivetran_deleted","_fivetran_synced","_fivetran_id_contactos","id_contactos",
    "contacto_owner_contactos","contacto_owner_name_contactos","created_time_contactos",
    "id_contacto_contactos","consentimento_contactos","id_lead_conversao_contactos",
    "data_de_recolha_de_consentimento_hyundai_contactos","chave_concessao_bd_rede_contactos",
    "chave_instalacao_bd_rede_contactos","chave_contrato_concessionario_bd_rede_contactos",
    "chave_agrupamento_performance_vendas_contactos","chave_agrupamento_cliente_contactos",
    "link_centro_consentimento_contactos","_fivetran_deleted_contactos","_fivetran_synced_contactos",
    "_fivetran_id_propostas","id_propostas","created_time_propostas",
    "proposta_realizada_owner_propostas","proposta_realizada_owner_name_propostas",
    "modified_time_propostas","conta_name_propostas","id_contacto_propostas","contacto_name_propostas",
    "deal_name_propostas","created_by_propostas","modified_by_propostas","descricao_apoio_propostas",
    "codigo_da_proposta_propostas","descricao_pintura_propostas","data_de_criacao_da_proposta_propostas",
    "data_de_entrega_da_proposta_propostas","id_conta_propostas","id_negocio_propostas",
    "id_proposta_realizada_propostas","data_de_validade_da_proposta_propostas","codigo_modelo_propostas",
    "sufixo_modelo_propostas","codigo_cor_exterior_propostas","codigo_cor_interior_propostas",
    "concessionario_owner_propostas","data_prevista_matricula_propostas","data_prevista_de_entrega_propostas",
    "data_da_conclusao_propostas","instalacao_propostas","chave_instalacao_bd_rede_propostas",
    "chave_concessao_bd_rede_propostas","cod_proposta_propostas","codigo_fabricante_cor_interior_propostas",
    "codigo_fabricante_cor_exterior_propostas","_fivetran_deleted_propostas","_fivetran_synced_propostas",
    "_fivetran_id_deals","deal_owner_name_deals","id_contacto_deals",
    "closing_date_deals","deal_owner_deals","conta_name_deals","contacto_name_deals","id_negocio_deals",
    "data_prevista_de_reentrada_em_negocio_deals","id_conta_deals","codigo_negocio_deals","data_venda_deals",
    "nome_contacto_deals","id_deals","data_decisao_negocio_deals","_fivetran_deleted_deals",
    "_fivetran_synced_deals","_fivetran_id_campaigns","id_campaigns",
    "campaign_owner_name_campaigns","created_time_campaigns","codigo_campanha_campaigns",
    "_fivetran_deleted_campaigns","_fivetran_synced_campaigns"
]

src_tbl = "sc_gold.Features_Table"

# compute keep list from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # quote names safely

# 2) Snapshot and recreate from projection
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")
spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")
spark.sql("DROP VIEW _src")

#You should generally remove columns that:
#Are unique identifiers (e.g., _fivetran_id, id, id_lead, id_contactos, id_propostas, id_deals, id_campaigns).
#Are timestamps or sync markers from ETL (_fivetran_synced, _fivetran_deleted, etc.).
#Contain names of people or owners (lead_owner_name, campaign_owner_name_campaigns, etc.).
#Are free-text codes or descriptive identifiers (codigo_campanha_campaigns, codigo_da_proposta_propostas, etc.).
#Are internal foreign keys / lookup fields (chave_instalacao_bd_rede, chave_agrupamento_cliente, etc.).
#Are emails / links (link_centro_consentimento_contactos).




In [0]:
# 1) Built the second delete list after a better analysis 
cols_to_delete = [
    # --- timestamps / dates ---
  
    "data_criacao_da_lead_contactos",
    "data_prevista_de_entrega_deals",
    "data_prevista_matricula_deals",
    "data_criacao_da_lead_deals",

    # --- IDs / keys ---
    "created_by", "modified_by",
    "id_classe_propostas", "id_model_group_propostas",
    "id_combustivel_propostas", "id_modelo_propostas",
    "chave_agrupamento_cliente_propostas",
    "tylacode_propostas",
    "chave_concessao_bd_rede_deals", "chave_instalacao_bd_rede_deals",
    "chave_contrato_concessionario_bd_rede_deals",
    "chave_agrupamento_performance_vendas_deals", "chave_agrupamento_cliente_deals","campanha_deals",

    # --- descriptive text ---
    "descricao_model_group_propostas",
    

    # --- administrative fields ---
     "gestor_area_propostas",

    # --- duplicated / rarely useful flags ---
    "hubleads__nao_comunica_com_cc_",
    "converted_contact", "converted_deal", "converted_from_lead_deals",
    "ccupdated_contactos", "myhyundai_contactos",
    "f2_deals", "modelovendaupdate_deals"
]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# 2) Snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN month(created_time) IN (12, 1, 2) THEN 'Winter'
    WHEN month(created_time) IN (3, 4, 5) THEN 'Spring'
    WHEN month(created_time) IN (6, 7, 8) THEN 'Summer'
    ELSE 'Autumn'
  END AS Season
FROM sc_gold.Features_Table;

--I created a Season for ML Models ( thinking in the prediction already)


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  datediff(created_time_deals, created_time) AS lead_to_deal_days
FROM sc_gold.Features_Table;

-- this is deals date- leads date to give the number of days that take , for my surprise there are negatives but can be typos or retroactive data entry → A deal was created manually before the lead was officially entered in the CRM.


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN datediff(created_time_deals, created_time) <= 7 THEN 'Fast'
    WHEN datediff(created_time_deals, created_time) <= 30 THEN 'Medium'
    ELSE 'Slow'
  END AS deal_speed_category
FROM sc_gold.Features_Table;



In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN created_time_deals < created_time THEN 1
    ELSE 0
  END AS deal_before_lead
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN dayofweek(created_time) IN (1,7) THEN 1
    ELSE 0
  END AS is_weekend_lead
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN day(created_time_deals) >= 25 THEN 1
    ELSE 0
  END AS is_month_end_deal
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table
SELECT 
    *,
    CASE 
        WHEN qtd_viaturas_deals = 0 OR qtd_viaturas_deals IS NULL 
        THEN 0
        ELSE num_de_propostas_deals / qtd_viaturas_deals
    END AS proposals_per_vehicle
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  year(created_time)  AS lead_year,
  month(created_time) AS lead_month
FROM sc_gold.Features_Table;


In [0]:
%sql
SELECT * FROM sc_gold.features_table

In [0]:
cols_to_delete = [
    "created_time",
    "activities_involved",
    "created_time_deals"
]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
%sql
SELECT
  is_converted,
  COUNT(*) AS total_leads
FROM sc_gold.leads_pbs
GROUP BY is_converted
ORDER BY is_converted;



In [0]:
# === Replace your cols_to_delete with this one (excludes columns you already removed) ===
cols_to_delete = [
    # ----------------------------
    # LEAKAGE (deal/post-outcome) — info not available at lead time
    # ----------------------------
    "lead_status",                    # leakage: deal-stage info
    "lead_status_2",                  # leakage: deal-stage variant
        "forecast_type_deals",                  # leakage
    "motivo_fecho_negocio_deals",           # leakage: reason for closing
     "categoria_negocio_venda_deals",        # leakage; also mostly null in many exports
    "modelo_de_venda_deals",                # leakage; often mostly null
    "tipo_de_negocio_deals",                # leakage; often mostly null
        "tipo_de_pedido",                 # leakage
         "tipo_de_pedido_contactos",                 # leakage
           "categoria_proposta_propostas",                       # leakage
           "activities_involved_deals",            # leakage: totals up to close
 

    # ----------------------------
    # ID-like / keys (non-predictive & high-unique)
    # ----------------------------
    "contrato_concessionario_deals",        # ID-like (deal)
    "contrato_concessionario",              # ID-like
    "contrato_concessionario_contactos",    # ID-like
    "concessionario_escolhido",             # ID-like / chosen dealer
    "estado_do_pedido_propostas",           # often key-like / admin-ish

    # ----------------------------
    # Data quality: free text / high null / near-constant / high-cardinality
    # ----------------------------
    "descricao_classe_propostas",           # free text / mostly null
    "subject_propostas",                    # free text / high-cardinality
    "margem_frota_eni_propostas",           # mostly null
    "sub_total_com_extras_propostas",       # mostly null
    "valor_aprovado_propostas",             # mostly null
    "valor_extras_propostas",               # mostly null
    "valor_do_apoio_pedido_propostas",      # mostly null; also key-like in some exports
    "n__matriculas_associadas_deals"        # near-constant / mostly null (deal-level)
]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
from pyspark.sql import functions as F

tbl = "sc_gold.Features_Table"
cols = [
    "concessao",
    "instalacao",
    "agrupamento_performance_vendas",
    "agrupamento_cliente",
    "concessao_contactos",
    "instalacao_contactos",
    "agrupamento_performance_vendas_contactos",
    "agrupamento_cliente_contactos",
    "concessao_propostas",
    "concessao_deals",
    "instalacao_deals",
    "agrupamento_performance_vendas_deals",
    "agrupamento_cliente_deals",
]

df = spark.table(tbl)
present = [c for c in cols if c in df.columns]
exprs = [
    (F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c)) for c in present
]
res = df.agg(*exprs).collect()[0].asDict()
total = df.count()

out = [{"column": c, "null_count": int(res.get(c, 0)), "rows_total": total,
        "null_pct": f"{(res.get(c, 0)/total)*100:.2f}%"} for c in present]
for c in cols:
    if c not in present:
        out.append({"column": c, "null_count": "—", "rows_total": total, "null_pct": "not in table"})

spark.createDataFrame(out).orderBy("column").show(truncate=False)


In [0]:
# === Replace your cols_to_delete with this one (excludes columns you already removed) ===
cols_to_delete = [
    # ----------------------------
    # LEAKAGE (deal/post-outcome) — info not available at lead time
    # ----------------------------
    "lead_status",                    # leakage: deal-stage info
    "instalacao",
    "agrupamento_performance_vendas",
    "agrupamento_cliente",
    "instalacao_contactos",
    "agrupamento_performance_vendas_contactos",
    "agrupamento_cliente_contactos",
    "concessao_propostas",
    "concessao_deals",
    "instalacao_deals",
    "agrupamento_performance_vendas_deals",
    "agrupamento_cliente_deals" ]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")

In [0]:
from pyspark.sql import functions as F

#since nulls are only 0.07%, filling with the mode (most frequent value) is a very good option.


col_name = "tipo_cliente_contactos"

# get mode (most frequent value)
mode_value = (
    df.groupBy(col_name)
      .count()
      .orderBy(F.desc("count"))
      .first()[0]
)

# fill nulls with mode
df = df.na.fill({col_name: mode_value})

# overwrite the existing table
df.write.mode("overwrite").saveAsTable("sc_gold.Features_Table")


In [0]:
from pyspark.sql import functions as F

# proposals_per_vehicle with small percentage of nulls (2.53%) we are going to use the median, is more robust.
#Fill with mean/median
#Median = more robust to outliers.

col_name = "proposals_per_vehicle"

# calculate median
median_val = df.approxQuantile(col_name, [0.5], 0.01)[0]  # 0.5 = median, tolerance=0.01

# print the median
print(f"Median value for {col_name}: {median_val}")

# fill nulls with median
df = df.na.fill({col_name: median_val})


In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

row_count = df.count()
dtypes = dict(df.dtypes)

# --- STRICT NULLS ONLY (what you asked) ---
null_aggs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
null_row = df.agg(*null_aggs).collect()[0].asDict()

rows = []
for c in df.columns:
    n_null = int(null_row.get(c, 0))
    rows.append((c, dtypes[c], n_null, row_count, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "dtype", "null_count", "rows_total", "null_pct"])
display(out.orderBy(F.desc("null_pct")))


In [0]:
# === Replace your cols_to_delete with this one (excludes columns you already removed) ===
cols_to_delete = [
    # ----------------------------
    # LEAKAGE (deal/post-outcome) — info not available at lead time
    # ----------------------------
    "lead_status",  
      "apoio_total_propostas",
    "forma_de_pagamento_propostas",
    "apoio_concessionario_propostas",
    "comparticipacao_retoma_concessao_propostas",
    "apoio_retoma_hyundai_propostas",
    "apoio_financiamento_cetelem_propostas",
    "formulario_deals",
    "comparticipacao_campanha_da_concessao_propostas",
    "lead_conversion_time_deals",
    "apoio_campanha_hyundai_propostas",
    "probability_____deals",
    "distrito",
    "apoio_hyundai_portugal_propostas",
    "valor_campanha_propostas",
    "desconto_total__c__apoio_de_importador__propostas",
    "categoria_negocio_deals",
        "distrito_contactos",
    "estado_ordering_propostas",
    "tasks_involved_deals",
    "events_involved_deals",
    "calls_involved_deals",
    "desconto_total_propostas",
                 
   ]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
CHECK_COLS = ["modelo", "modelo_contactos", "modelo_propostas", "modelos_deals"]

df = spark.table(TBL)
present = [c for c in CHECK_COLS if c in df.columns]
missing = [c for c in CHECK_COLS if c not in df.columns]

row_count = df.count()
# null counts for present columns
nulls = (df.agg(*[F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in present])
           .collect()[0]
           .asDict())

# build result rows
rows = []
for c in present:
    n_null = int(nulls.get(c, 0))
    rows.append((c, row_count, n_null, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "rows_total", "null_count", "null_pct"])
display(out.orderBy("column"))

if missing:
    print("Not found in table (skipped):", missing)


In [0]:
# === Replace your cols_to_delete with this one (excludes columns you already removed) ===
cols_to_delete = [
    # ----------------------------
    # LEAKAGE (deal/post-outcome) — info not available at lead time
    # ----------------------------
    
    "modelo_propostas",
    "modelos_deals", 
    "formulario",
    "tasks_only_deals",
    "events_only_deals",
    "calls_only_deals",
    "tasks_involved",
    "events_involved",
    "calls_involved",
               
   ]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

row_count = df.count()
dtypes = dict(df.dtypes)

# --- STRICT NULLS ONLY (what you asked) ---
null_aggs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
null_row = df.agg(*null_aggs).collect()[0].asDict()

rows = []
for c in df.columns:
    n_null = int(null_row.get(c, 0))
    rows.append((c, dtypes[c], n_null, row_count, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "dtype", "null_count", "rows_total", "null_pct"])
display(out.orderBy(F.desc("null_pct")))

In [0]:
# === Replace your cols_to_delete with this one ===
cols_to_delete = [
    # ----------------------------
    # DEAL / POST-OUTCOME LEAKAGE  (*_deals)
    # ----------------------------
    "formulario_contactos",
    "is_converted",
    "tasks_involved",
    "events_involved",
    "calls_involded",
    "tasks_involved_contactos",
    "events_involved_contactos",
    "calls_involded_contactos",
    "activities_involved_contactos",
    "email_opt_out_contactos",
       "pais_contactos",
            "tipo_cliente_propostas",
      "isv_propostas",
    "pintura_propostas",
    "base_tributavel_propostas",
    "legalizacao_propostas",
    "iuc_propostas",
    "preco_total_propostas",
    "iva_propostas",
    "sub_total_com_desconto_propostas",
    "taxa_propostas",
    "total_final_propostas",
    "s_g_p_u__propostas",
     "tasks_only_deals",
    "events_only_deals",
    "calls_only_deals",
    "caracterizacao_contactos",
        "caracterizacao_deals",
    "n__matriculas_por_associar_deals",
    "tipo_cliente_negocio_deals",
    "sales_cycle_duration_deals",
    "calls_involved_contactos",
    "concessao",
    "origem_contactos",
    "valor_base_propostas",
    "preco_base_propostas",
    "age_tier_deals",
    "num_de_propostas_deals",
    "qtd_viaturas_propostas",
    "qtd_viaturas_deals",
    "age_in_days_deals",
    "modelo",
]
 


src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")
spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")
spark.sql("DROP VIEW _src")


In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

row_count = df.count()
dtypes = dict(df.dtypes)

# --- STRICT NULLS ONLY (what you asked) ---
null_aggs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
null_row = df.agg(*null_aggs).collect()[0].asDict()

rows = []
for c in df.columns:
    n_null = int(null_row.get(c, 0))
    rows.append((c, dtypes[c], n_null, row_count, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "dtype", "null_count", "rows_total", "null_pct"])
display(out.orderBy(F.desc("null_pct")))

In [0]:
%sql

SELECT * FROM sc_gold.features_table

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

row_count = df.count()
dtypes = dict(df.dtypes)

# --- STRICT NULLS ONLY (what you asked) ---
null_aggs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
null_row = df.agg(*null_aggs).collect()[0].asDict()

rows = []
for c in df.columns:
    n_null = int(null_row.get(c, 0))
    rows.append((c, dtypes[c], n_null, row_count, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "dtype", "null_count", "rows_total", "null_pct"])
display(out.orderBy(F.desc("null_pct")))

In [0]:
from pyspark.sql import functions as F

TABLE = "sc_gold.features_table"
DIFF_COL = "lead_to_deal_days"

# Load table
sdf = spark.table(TABLE)

# ---- Counts ----
total = sdf.count()
neg_count = sdf.filter(F.col(DIFF_COL) < 0).count()
pos_count = total - neg_count
neg_pct = round(100 * neg_count / total, 1) if total else 0.0
pos_pct = round(100 * pos_count / total, 1) if total else 0.0

print(f"Total rows: {total}")
print(f"Positive count: {pos_count} ({pos_pct}%)")
print(f"Negative count: {neg_count} ({neg_pct}%)")

# ---- Distribution buckets for positives ----
pos = (
    sdf.filter(F.col(DIFF_COL) >= 0)
       .withColumn(
           "pos_bucket",
           F.when(F.col(DIFF_COL) < 1, "0–1d")
            .when(F.col(DIFF_COL) < 7, "1–7d")
            .when(F.col(DIFF_COL) < 30, "7–30d")
            .when(F.col(DIFF_COL) < 90, "30–90d")
            .when(F.col(DIFF_COL) < 180, "90–180d")
            .when(F.col(DIFF_COL) < 365, "180–365d")
            .otherwise(">365d")
       )
)
print("\nPositive distribution by bucket:")
pos.groupBy("pos_bucket").count().orderBy("pos_bucket").show(truncate=False)

# ---- Distribution buckets for negatives ----
neg = (
    sdf.filter(F.col(DIFF_COL) < 0)
       .withColumn(
           "neg_bucket",
           F.when(F.col(DIFF_COL) >= -1, "<1d early")
            .when(F.col(DIFF_COL) >= -7, "1–7d early")
            .when(F.col(DIFF_COL) >= -30, "7–30d early")
            .when(F.col(DIFF_COL) >= -90, "30–90d early")
            .when(F.col(DIFF_COL) >= -180, "90–180d early")
            .when(F.col(DIFF_COL) >= -365, "180–365d early")
            .otherwise(">365d early")
       )
)
print("\nNegative distribution by bucket:")
neg.groupBy("neg_bucket").count().orderBy("neg_bucket").show(truncate=False)



In [0]:
# Databricks / PySpark + matplotlib

from pyspark.sql import functions as F
import matplotlib.pyplot as plt

TABLE   = "sc_gold.features_table"
DIFF_COL = "lead_to_deal_days"

# 1) Load and keep only numeric, non-null diffs
sdf = spark.table(TABLE).select(F.col(DIFF_COL).cast("double").alias("diff_days")).na.drop()

# 2) Counts / percentages
total = sdf.count()
neg_count = sdf.filter("diff_days < 0").count()
pos_count = total - neg_count

neg_pct = round(100 * neg_count / total, 1) if total else 0.0
pos_pct = round(100 * pos_count / total, 1) if total else 0.0

print(f"Total rows: {total}")
print(f"Positive (>=0) count: {pos_count}  ({pos_pct}%)")
print(f"Negative (<0) count: {neg_count}  ({neg_pct}%)")

# 3) Bring to Pandas for plotting
pdf = sdf.toPandas()

# 4) Distribution graph (two histograms overlaid)
plt.figure(figsize=(10,5))
plt.hist(pdf.loc[pdf["diff_days"]>=0, "diff_days"], bins=60, alpha=0.7, label="Positives (deal after lead)")
plt.hist(pdf.loc[pdf["diff_days"]<0,  "diff_days"], bins=60, alpha=0.7, label="Negatives (deal before lead)")
plt.xlabel("Days difference (deal_created - lead_created)")
plt.ylabel("Number of records")
plt.title("Distribution of lead_to_deal_days (positives vs negatives)")
plt.legend()
plt.tight_layout()
plt.show()

# (Optional) Zoomed view for 0–100 days to see the main cluster clearly
plt.figure(figsize=(10,5))
subset = pdf[(pdf["diff_days"] >= 0) & (pdf["diff_days"] <= 100)]
plt.hist(subset["diff_days"], bins=50, alpha=0.9)
plt.xlabel("Days difference (0–100 days)")
plt.ylabel("Number of records")
plt.title("Positive differences (0–100 days)")
plt.tight_layout()
plt.show()


In [0]:
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

TABLE = "sc_gold.features_table"
DIFF_COL = "lead_to_deal_days"

# Load only negatives
neg = spark.table(TABLE).select(F.col(DIFF_COL).cast("double").alias("diff_days")) \
         .filter("diff_days < 0").na.drop()

# Count how many
neg_count = neg.count()
print(f"Negative rows: {neg_count}")

# Bring to pandas for plotting
pdf_neg = neg.toPandas()

# Histogram for negatives
plt.figure(figsize=(10,5))
plt.hist(pdf_neg["diff_days"], bins=50, alpha=0.8, color="red")
plt.xlabel("Days difference (deal_created - lead_created)")
plt.ylabel("Number of records")
plt.title("Distribution of NEGATIVE lead_to_deal_days (deal before lead)")
plt.tight_layout()
plt.show()


In [0]:
%sql
SELECT 
    deal_before_lead,
    COUNT(*) AS count,
    ROUND(100 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM sc_gold.features_table
GROUP BY deal_before_lead
ORDER BY deal_before_lead;


In [0]:
# Remove invalid rows where deal happens before lead
# These cases have no operational meaning, so we delete them

src_tbl = "sc_gold.features_table"

# Read the table
df = spark.table(src_tbl)

# Filter out rows where deal_before_lead = 1
df_filtered = df.filter(df.deal_before_lead == 0)

# Drop the 'deal_before_lead' column (no longer needed)
df_filtered = df_filtered.drop("deal_before_lead")

# Overwrite the existing table with the cleaned data
df_filtered.write.mode("overwrite").saveAsTable(src_tbl)

print("✅ Rows with deal_before_lead = 1 removed and column dropped successfully.")


In [0]:
# === Drop only the 'deal_before_lead' column ===

src_tbl = "sc_gold.features_table"

# Define the column to delete
cols_to_delete = ["deal_before_lead","pais"]

# Get the full list of columns dynamically
all_cols = spark.table(src_tbl).columns

# Keep everything except the column to delete
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# Snapshot current table into a temp view
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

# Overwrite the table without the dropped column
spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql}
FROM _src
""")

# Drop the temporary view
spark.sql("DROP VIEW _src")

print("✅ Column 'deal_before_lead' dropped successfully from table.")


In [0]:
%sql

SELECT * FROM sc_gold.features_table

In [0]:
# ===============================================================
# Filter dataset to include only records with a filled 'stage_deals'
# Ensures proposals that had follow-up toward a deal are kept
# ===============================================================

src_tbl = "sc_gold.features_table"

# Read the table
df = spark.table(src_tbl)

# Filter only rows with non-null 'stage_deals'
df_filtered = df.filter(df.stage_deals.isNotNull())

# Overwrite the existing table with the filtered data
df_filtered.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(src_tbl)

print("✅ Filter applied successfully — kept only rows with non-null 'stage_deals'.")


In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

row_count = df.count()
dtypes = dict(df.dtypes)

# --- STRICT NULLS ONLY (what you asked) ---
null_aggs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
null_row = df.agg(*null_aggs).collect()[0].asDict()

rows = []
for c in df.columns:
    n_null = int(null_row.get(c, 0))
    rows.append((c, dtypes[c], n_null, row_count, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "dtype", "null_count", "rows_total", "null_pct"])
display(out.orderBy(F.desc("null_pct")))
display(out, truncate=False)


In [0]:
from pyspark.sql import functions as F

src_tbl = "sc_gold.features_table"
df = spark.table(src_tbl)

# Normalize common variants for stage labels (optional – add/remove as needed)
venda_labels  = ['Vendido', 'Venda-aprovado', 'Venda Aprovado', 'Venda Aprovada']
emcurso_labels = ['Em Curso', 'Em curso']
adiado_labels  = ['Adiado']

# Fill only when motivo_da_perda_deals is NULL
filled_col = (
    F.when(
        F.col("motivo_da_perda_deals").isNull() & F.col("stage_deals").isin(*venda_labels),
        F.lit("Não aplicável (venda)")
    )
    .when(
        F.col("motivo_da_perda_deals").isNull() & F.col("stage_deals").isin(*emcurso_labels),
        F.lit("Sem decisão (em curso)")
    )
    .when(
        F.col("motivo_da_perda_deals").isNull() & F.col("stage_deals").isin(*adiado_labels),
        F.lit("Sem decisão (adiado)")
    )
    .otherwise(F.col("motivo_da_perda_deals"))
)

df_out = df.withColumn("motivo_da_perda_deals", filled_col)

# Overwrite the table with the updated values
df_out.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(src_tbl)

print("✅ Filled 'motivo_da_perda_deals' conditionally based on 'stage_deals'.")


In [0]:
from pyspark.sql import functions as F

src_tbl = "sc_gold.features_table"
df = spark.table(src_tbl)

# Create binary flag: 1 if model name exists in campaign name, else 0
df = df.withColumn(
    "model_in_campaign",
    F.when(
        F.lower(F.col("campaign_name_campaigns")).contains(F.lower(F.col("modelo_contactos"))),
        F.lit(1)
    ).otherwise(F.lit(0))
)

# Overwrite table with new column
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(src_tbl)

print("✅ Column 'model_in_campaign' added successfully (1 = model appears in campaign).")


In [0]:
%sql
SELECT * FROM sc_gold.features_table

In [0]:
%sql
SELECT 
    modelo_contactos,
    campaign_name_campaigns,
    model_in_campaign
FROM sc_gold.features_table;


In [0]:
from pyspark.sql import functions as F

src_tbl = "sc_gold.features_table"
df = spark.table(src_tbl)

# === Calculate median pvp_propostas by modelo_contactos ===
median_by_model = (
    df.groupBy("modelo_contactos")
      .agg(F.expr("percentile_approx(pvp_propostas, 0.5)").alias("median_pvp"))
)

# Join the median back to the main dataframe
df = df.join(median_by_model, on="modelo_contactos", how="left")

# Fill nulls in pvp_propostas with the model-specific median
df = df.withColumn(
    "pvp_propostas",
    F.when(F.col("pvp_propostas").isNull(), F.col("median_pvp"))
     .otherwise(F.col("pvp_propostas"))
)

# Drop the helper median column
df = df.drop("median_pvp")

# Save updates
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(src_tbl)

print("✅ Filled nulls in 'pvp_propostas' with model-specific median values.")


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window  # ✅ this import was missing

# Define source table
src_tbl = "sc_gold.features_table"
df = spark.table(src_tbl)

# === 1. Calculate mode 'combustivel_propostas' by 'modelo_contactos' ===
# Get the most frequent fuel type per model
mode_by_model = (
    df.groupBy("modelo_contactos", "combustivel_propostas")
      .count()
      .withColumn("rn", F.row_number().over(
          Window.partitionBy("modelo_contactos").orderBy(F.desc("count"))
      ))
      .filter(F.col("rn") == 1)
      .select("modelo_contactos", "combustivel_propostas")
      .withColumnRenamed("combustivel_propostas", "mode_combustivel")
)

# === 2. Join mode back to main dataframe ===
df = df.join(mode_by_model, on="modelo_contactos", how="left")

# === 3. Fill nulls in 'combustivel_propostas' with the model-specific mode ===
df = df.withColumn(
    "combustivel_propostas",
    F.when(F.col("combustivel_propostas").isNull(), F.col("mode_combustivel"))
     .otherwise(F.col("combustivel_propostas"))
)

# === 4. Drop helper column ===
df = df.drop("mode_combustivel")

# === 5. Save updated table ===
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(src_tbl)

print("✅ Filled nulls in 'combustivel_propostas' using model-specific mode (by 'modelo_contactos').")



In [0]:
%sql
SELECT *,
    CASE 
        WHEN salutation_contactos IN ('Sr', 'Sr.', 'Dr.', 'Eng.', 'Prof.') THEN 1
        WHEN salutation_contactos IN ('Sr.ª', 'Dra.', 'Dr.ª') THEN 0
        ELSE NULL
    END AS is_male
FROM sc_gold.features_table


In [0]:
%sql
UPDATE sc_gold.features_table
SET is_male = CASE
  WHEN salutation_contactos IN ('Sr','Sr.','Dr.','Eng.','Prof.') THEN 1
  WHEN salutation_contactos IN ('Sr.ª','Dra.','Dr.ª') THEN 0
  ELSE NULL
END;


In [0]:
%sql
SELECT 
    salutation_contactos,
    CASE 
        WHEN salutation_contactos IN ('Sr', 'Sr.', 'Dr.', 'Eng.', 'Prof.') THEN 1
        WHEN salutation_contactos IN ('Sr.ª', 'Dra.', 'Dr.ª') THEN 0
        ELSE NULL
    END AS is_male
FROM sc_gold.features_table


In [0]:
# === Drop only the 'deal_before_lead' column ===

src_tbl = "sc_gold.features_table"

# Define the column to delete
cols_to_delete = ["salutation_contactos","overall_sales_duration_deals"]

# Get the full list of columns dynamically
all_cols = spark.table(src_tbl).columns

# Keep everything except the column to delete
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# Snapshot current table into a temp view
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

# Overwrite the table without the dropped column
spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql}
FROM _src
""")

# Drop the temporary view
spark.sql("DROP VIEW _src")

print("✅ Column 'salutation_contactos' dropped successfully from table.")

In [0]:
%sql
SELECT * FROM sc_gold.features_table

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

row_count = df.count()
dtypes = dict(df.dtypes)

# --- STRICT NULLS ONLY (what you asked) ---
null_aggs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
null_row = df.agg(*null_aggs).collect()[0].asDict()

rows = []
for c in df.columns:
    n_null = int(null_row.get(c, 0))
    rows.append((c, dtypes[c], n_null, row_count, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "dtype", "null_count", "rows_total", "null_pct"])
display(out.orderBy(F.desc("null_pct")))
display(out, truncate=False)


In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (no column exclusions, no saving) ===
import time, numpy as np, pandas as pd, sklearn
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Prepare types (use ALL columns)
# - numeric: numbers + booleans (treated as numeric 0/1)
# - datetimes: convert to epoch seconds (new *_ts numeric columns)
# - everything else: categorical (OHE)

# Detect native datetime dtypes and convert to numeric copies
date_cols = pdf.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns.tolist()
for c in date_cols:
    pdf[f"{c}_ts"] = pd.to_datetime(pdf[c], errors="coerce").view("int64") / 1e9  # seconds
# Optional: try to catch obvious date-like strings (kept light to avoid surprises)
# (commented; enable if needed)
# for c in pdf.select_dtypes(include=["object"]).columns:
#     try_dt = pd.to_datetime(pdf[c], errors="coerce", utc=False)
#     if try_dt.notna().sum() > 0 and (try_dt.notna().mean() > 0.8):
#         pdf[f"{c}_ts"] = try_dt.view("int64") / 1e9

# Build column groups
num_cols = pdf.select_dtypes(include=["number", "bool"]).columns.tolist()
cat_cols = pdf.select_dtypes(include=["object", "category"]).columns.tolist()

print(f"Detected -> numeric={len(num_cols)}, categorical={len(cat_cols)}, datetime={len(date_cols)} (converted to *_ts)")

# 3) Preprocess (no exclusions)
ohe_kw = {"handle_unknown": "ignore"}
# sklearn ≥1.2 uses 'sparse_output' arg, older uses 'sparse'
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kw["sparse_output"] = True
else:
    ohe_kw["sparse"] = True

transformers = []
if len(num_cols):
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if len(cat_cols):
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(**ohe_kw))
    ]), cat_cols))

if not transformers:
    raise ValueError("No usable columns: table has neither numeric/boolean nor categorical fields.")

preprocess = ColumnTransformer(transformers=transformers, remainder="drop", sparse_threshold=1.0)

# 4) Encode -> SVD (<=20) -> Standardize
Xs = preprocess.fit_transform(pdf)  # sparse (likely) after OHE
n_comp = max(2, min(20, Xs.shape[1] - 1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
X = StandardScaler().fit_transform(X)

print(f"Encoded shape: {Xs.shape} | Reduced (SVD): {X.shape}")

# 5) k-scan (silhouette + inertia) and pick best k
k_range = range(2, min(9, X.shape[0]))  # guard for tiny samples
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  silhouette={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.2f}s")
print(f"Total k-scan time: {time.perf_counter() - t0:.2f}s")

import pandas as pd
metrics_df = pd.DataFrame(metrics, columns=["k","silhouette","inertia","sec_per_fit"])
best_row = metrics_df.sort_values(["silhouette","k"], ascending=[False, True]).iloc[0]
best_k = int(best_row.k)
print("\nK-scan (sorted by silhouette):\n", metrics_df.sort_values(["silhouette","k"], ascending=[False, True]).to_string(index=False))
print(f"\n>>> Best k = {best_k}  (silhouette={best_row.silhouette:.4f})")

# 6) Final model + attach labels (in-memory only)
final_km = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = final_km.labels_

# 7) Quick summary
sizes = pdf["cluster"].value_counts(dropna=False).sort_index().to_frame("count")
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)
print("\nCluster sizes:\n", sizes)

# Optional: preview by cluster
print("\nPreview (first 5 rows with cluster):")
print(pdf.head(5)[["cluster"] + [c for c in pdf.columns if c != "cluster"]].to_string(index=False))


In [0]:
# ==== CLUSTER PROFILING & VISUALS (drop-in, run after your K-Means cell) ====
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_samples

# ---------- Helpers ----------
def quick_readout(df, cluster_col, value_col, buckets=(30, 60, 90), unit="d"):
    """
    Print median and cumulative bucket percentages (<=b1, <=b2, >b_last) for a numeric KPI.
    """
    if value_col not in df.columns:
        print(f"[quick_readout] Column '{value_col}' not found.")
        return
    print("=== Quick readout per cluster ===")
    for k in sorted(df[cluster_col].dropna().unique()):
        g = df[df[cluster_col] == k][value_col].astype(float)
        if g.empty:
            print(f"Cluster {k}: (no rows)")
            continue
        med = np.nanmedian(g)
        pcs = []
        for b in buckets:
            pcs.append((f"<= {b}{unit}", (g <= b).mean() * 100))
        pcs.append((f"> {buckets[-1]}{unit}", (g > buckets[-1]).mean() * 100))
        pcs_str = "  ".join([f"{lab}={pct:.1f}%" for lab, pct in pcs])
        print(f"Cluster {k}: median={med:.1f}{unit}  |  {pcs_str}")

def top_cats(df, cluster_col, cat_col, topn=5):
    """
    Top-N categories inside each cluster for one categorical column.
    Coerces values to safe strings to avoid Arrow conversion errors.
    """
    out_rows = []
    for k, g in df.groupby(cluster_col):
        s = g[cat_col].astype("object").where(lambda x: x.notna(), other="<<NULL>>")
        s = s.map(lambda x: str(x))
        s = s.map(lambda x: x if len(x) <= 200 else (x[:197] + "..."))
        vc = s.value_counts(normalize=True, dropna=False).head(topn)
        for val, frac in vc.items():
            out_rows.append([cat_col, int(k), val, float(round(frac*100, 2))])
    top = pd.DataFrame(out_rows, columns=["column", "cluster", "value", "pct"])
    return top.astype({"column":"string","value":"string","cluster":"int64","pct":"float64"})

# ---------- 1) Cluster sizes ----------
sizes = pdf["cluster"].value_counts(dropna=False).sort_index()
sizes_df = sizes.to_frame("count").assign(pct=(sizes / sizes.sum() * 100).round(2))
display(sizes_df)

plt.figure()
sizes.plot(kind="bar")
plt.title("Cluster sizes"); plt.xlabel("cluster"); plt.ylabel("count")
plt.show()

# ---------- 2) Quick readout for your key KPI (edit the column name if needed) ----------
target_numeric = "lead_to_deal_days"  # <- change this KPI if you want
if target_numeric in pdf.columns:
    quick_readout(pdf, "cluster", target_numeric, buckets=(30, 60, 90), unit="d")
else:
    print(f"[info] '{target_numeric}' not in table; skipping quick readout.")

# ---------- 3) Numeric profile (median/mean/std) ----------
num_cols = [c for c in pdf.select_dtypes(include=["number", "bool"]).columns if c != "cluster"]
if num_cols:
    prof_num = pdf.groupby("cluster")[num_cols].agg(["median", "mean", "std"])
    prof_num.columns = [f"{c}_{stat}" for c, stat in prof_num.columns]
    display(prof_num)

    # Heatmap of standardized medians (z-score) — slide-friendly
    med = pdf.groupby("cluster")[num_cols].median(numeric_only=True)
    med_std = (med - med.mean()) / med.std(ddof=0)
    keep = med_std.columns[:15]  # show first 15 numerics for readability
    plt.figure(figsize=(min(12, 0.6*len(keep)+3), 0.5*len(med_std)+3))
    plt.imshow(med_std[keep], aspect="auto")
    plt.yticks(range(len(med_std.index)), med_std.index)
    plt.xticks(range(len(keep)), keep, rotation=60, ha="right")
    plt.title("Standardized medians by cluster (z-score)")
    plt.colorbar(); plt.tight_layout(); plt.show()
else:
    print("[info] No numeric columns to profile.")

# ---------- 4) Top categories per cluster (robust) ----------
cat_cols = pdf.select_dtypes(include=["object", "category"]).columns.tolist()
if cat_cols:
    frames = [top_cats(pdf, "cluster", c, topn=5) for c in cat_cols]
    topcats = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["column","cluster","value","pct"])
    # If 'display' throws Arrow errors in your workspace, use print(topcats.to_string(index=False)) instead
    display(topcats)
else:
    print("[info] No categorical columns detected.")

# ---------- 5) 2-D projection scatter (SVD on preprocessed features) ----------
# Uses existing Xs (sparse encoded matrix) if available; falls back to X if that exists.
try:
    if "Xs" in globals():
        svd2 = TruncatedSVD(n_components=2, random_state=42)
        X2 = svd2.fit_transform(Xs)  # type: ignore
    elif "X" in globals():
        # If you only have the reduced matrix, reuse it (pad to 2D if needed)
        X2 = X if X.shape[1] >= 2 else np.pad(X, ((0,0),(0,2-X.shape[1])), mode="constant")
    else:
        raise NameError("Neither Xs nor X found")
    plt.figure(figsize=(6,5))
    for k in sorted(pdf["cluster"].unique()):
        idx = (pdf["cluster"] == k).values
        plt.scatter(X2[idx,0], X2[idx,1], s=8, alpha=0.6, label=f"c{k}")
    plt.legend(title="cluster", ncol=2)
    plt.title("2-D projection colored by cluster")
    plt.xlabel("Comp 1"); plt.ylabel("Comp 2")
    plt.tight_layout(); plt.show()
except Exception as e:
    print("[scatter] Skipped 2-D projection:", e)

# ---------- 6) Silhouette distribution per cluster ----------
try:
    if "X" in globals():
        sil = silhouette_samples(X, pdf["cluster"].values)
        tmp = pd.DataFrame({"cluster": pdf["cluster"].values, "silhouette": sil})
        ax = tmp.boxplot(by="cluster", column="silhouette", grid=False, figsize=(6,4))
        plt.suptitle("")
        plt.title("Silhouette by cluster")
        plt.xlabel("cluster"); plt.ylabel("silhouette")
        plt.show()
    else:
        print("[silhouette] Skipped (matrix X not found).")
except Exception as e:
    print("[silhouette] Skipped due to error:", e)


In [0]:
# metrics_df already built earlier as: columns=["k","silhouette","inertia","sec_per_fit"]
# 1) Print best k and silhouette (overall)
best_row = metrics_df.sort_values(["silhouette","k"], ascending=[False, True]).iloc[0]
best_k = int(best_row["k"])
best_sil = float(best_row["silhouette"])
print(f"Best k = {best_k}  |  silhouette = {best_sil:.4f}")

# 2) Elbow curve (inertia) and silhouette vs k
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)")
ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)")
ax.set_title("Silhouette vs k")
plt.show()


In [0]:
# === Best-practice K-Means (DECIMAL→DOUBLE without duplicates, car value kept, robust profiling) ===
import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn
from packaging import version
from pyspark.sql import functions as F
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, silhouette_samples

warnings.filterwarnings("ignore", category=UserWarning, module="threadpoolctl")

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# ---------- 1) Load Spark DF and CAST DECIMALs to DOUBLE *replacing* originals (avoid duplicates) ----------
cast_map = {
    "pvp_propostas": "DOUBLE",
    "proposals_per_vehicle": "DOUBLE",
    "lead_to_deal_days": "DOUBLE",
    "overall_sales_duration_deals": "DOUBLE",
}
sdf = spark.table(SRC_TBL)
cast_cols  = [F.col(c).cast(t).alias(c) for c, t in cast_map.items()]
other_cols = [F.col(c) for c in sdf.columns if c not in cast_map]
sdf_cast = sdf.select(*(cast_cols + other_cols))  # replaces originals, no duplicates
pdf = sdf_cast.toPandas()
pdf = pdf.loc[:, ~pdf.columns.duplicated()]       # belt-and-suspenders
print("Rows:", len(pdf), "| Cols:", len(pdf.columns))
existing = set(pdf.columns)

# ---------- 2) Feature sets ----------
# Strong numerics (log-scale the heavy-tailed ones)
log_num_desired = ["pvp_propostas", "lead_to_deal_days", "overall_sales_duration_deals"]
lin_num_desired = ["proposals_per_vehicle"]

# Compact behavior categoricals
cat_cols_desired = [
    "lead_source", "deal_speed_category", "stage_deals",
    "tipo_cliente_contactos", "combustivel_propostas", "origem_do_negocio_deals"
]

# Skip noisy/redundant (modeling only)
manual_drop = [
    "concessao_contactos", "versao_propostas", "campaign_name_campaigns",
    "modelo_contactos", "proposta_realizada_stage_propostas", "motivo_da_perda_deals"
]

# Keep only those that exist
log_num = [c for c in log_num_desired if c in existing]
lin_num = [c for c in lin_num_desired if c in existing]
cat_cols = [c for c in cat_cols_desired if c in existing]
drop_cols = [c for c in manual_drop if c in existing]
Xdf = pdf.drop(columns=drop_cols)
print("Numerics log-scaled:", log_num)
print("Numerics linear:", lin_num)
print("Categoricals:", cat_cols)
if drop_cols: print("Dropped (modeling only):", drop_cols)
assert (log_num or lin_num or cat_cols), "No usable features."

# ---------- 3) Preprocess (median impute; __MISSING__ bucket for cats; rare bucketing if available) ----------
def _log1p(x): return np.log1p(x)

ohe_kw = {"handle_unknown": "ignore"}
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kw["sparse_output"] = True
else:
    ohe_kw["sparse"] = True
ohe_extra = {"min_frequency": 0.02, "max_categories": 25} if version.parse(sklearn.__version__) >= version.parse("1.1") else {}

stages = []
if log_num:
    stages.append((
        "num_log",
        Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("log", FunctionTransformer(_log1p, feature_names_out="one-to-one")),
            ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
        ]),
        log_num
    ))
if lin_num:
    stages.append((
        "num_lin",
        Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
        ]),
        lin_num
    ))
if cat_cols:
    stages.append((
        "cat",
        Pipeline([
            ("imp", SimpleImputer(strategy="constant", fill_value="__MISSING__")),
            ("ohe", OneHotEncoder(**ohe_kw, **ohe_extra)),
        ]),
        cat_cols
    ))

preprocess = ColumnTransformer(stages, remainder="drop", sparse_threshold=1.0)

# ---------- 4) Encode -> SVD (12 comps) -> whiten ----------
Xs = preprocess.fit_transform(Xdf)                        # sparse matrix after OHE
n_comp = max(2, min(12, Xs.shape[1]-1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
expl = svd.explained_variance_ratio_.sum()
X = StandardScaler().fit_transform(X)                    # final whitening for K-Means
print(f"Sparse shape: {Xs.shape} | Reduced: {X.shape} | SVD explained variance: {expl:.3f}")

# ---------- 5) k-scan (Elbow + Silhouette) ----------
metrics = []
for k in range(2, 10):
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=30, max_iter=250, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_)))
    print(f"k={k} | silhouette={sil:.4f} | inertia={km.inertia_:.0f}")

m = pd.DataFrame(metrics, columns=["k","silhouette","inertia"])
best = m.sort_values(["silhouette","k"], ascending=[False, True]).iloc[0]
best_k, best_sil = int(best.k), float(best.silhouette)
print(f"\nBest k = {best_k} | silhouette = {best_sil:.4f}")

fig, ax = plt.subplots()
ax.plot(m.k, m.inertia, marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)"); ax.set_title("Elbow Curve (Inertia)")
plt.show()

fig, ax = plt.subplots()
ax.plot(m.k, m.silhouette, marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)"); ax.set_title("Silhouette vs k")
plt.show()

# ---------- 6) Final model + labels (in-memory only) ----------
final = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=60, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = final.labels_

# ---------- 7) Profiling ----------

def quick_readout(df, cluster_col, value_col, buckets=(30, 60, 90), unit="d"):
    if value_col not in df.columns:
        print(f"[quick_readout] '{value_col}' not found."); return
    print("=== Quick readout per cluster ===")
    for k in sorted(df[cluster_col].dropna().unique()):
        g = df[df[cluster_col]==k][value_col].astype(float)
        if g.empty: print(f"Cluster {k}: (no rows)"); continue
        med = np.nanmedian(g)
        parts = [(f"<= {b}{unit}", (g<=b).mean()*100) for b in buckets] + [(f"> {buckets[-1]}{unit}", (g>buckets[-1]).mean()*100)]
        print(f"Cluster {k}: median={med:.1f}{unit} | " + "  ".join([f"{a}={b:.1f}%" for a,b in parts]))

def top_cats(df, cluster_col, cat_col, topn=5):
    out=[]
    for k,g in df.groupby(cluster_col):
        s=(g[cat_col].astype("object").where(lambda x: x.notna(),"__MISSING__")
           .map(str).map(lambda x: x if len(x)<=200 else x[:197]+"..."))
        vc=s.value_counts(normalize=True, dropna=False).head(topn)
        for val, frac in vc.items(): out.append([cat_col,int(k),val,float(round(frac*100,2))])
    res=pd.DataFrame(out, columns=["column","cluster","value","pct"])
    return res.astype({"column":"string","value":"string","cluster":"int64","pct":"float64"})

# Sizes
sizes = pdf["cluster"].value_counts(dropna=False).sort_index()
display(sizes.to_frame("count").assign(pct=(sizes/len(pdf)*100).round(2)))

# Quick KPI readouts
if "lead_to_deal_days" in pdf.columns:
    quick_readout(pdf, "cluster", "lead_to_deal_days", buckets=(30,60,90), unit="d")

if "pvp_propostas" in pdf.columns:
    med_price = pdf.groupby("cluster")["pvp_propostas"].median()
    print("\nMedian car value (pvp_propostas) by cluster:\n", med_price)

# Numeric heatmap (only model numerics; guard for dtype)
num_for_prof = [c for c in (log_num + lin_num) if c in pdf.columns]
if num_for_prof:
    med = pdf.groupby("cluster")[num_for_prof].median(numeric_only=True)
    cols = [c for c in num_for_prof if c in med.columns]
    if cols:
        med_std = (med[cols] - med[cols].mean()) / med[cols].std(ddof=0)
        plt.figure(figsize=(min(12, 0.6*len(cols)+3), 0.5*len(med_std)+3))
        plt.imshow(med_std[cols], aspect="auto")
        plt.yticks(range(len(med_std.index)), med_std.index)
        plt.xticks(range(len(cols)), cols, rotation=60, ha="right")
        plt.title("Cluster z-score profiles (numeric medians)")
        plt.colorbar(); plt.tight_layout(); plt.show()

# Top categories per cluster (only the ones used in the model)
if cat_cols:
    topcats = pd.concat([top_cats(pdf, "cluster", c, topn=5) for c in cat_cols], ignore_index=True)
    display(topcats)

# 2-D projection & silhouette by cluster
try:
    svd2 = TruncatedSVD(n_components=2, random_state=SEED)
    X2 = svd2.fit_transform(Xs)
    plt.figure(figsize=(6,5))
    for k in sorted(pdf["cluster"].unique()):
        idx = (pdf["cluster"]==k).values
        plt.scatter(X2[idx,0], X2[idx,1], s=8, alpha=0.6, label=f"c{k}")
    plt.legend(title="cluster", ncol=2); plt.title("2-D projection (SVD) by cluster")
    plt.xlabel("Comp 1"); plt.ylabel("Comp 2"); plt.tight_layout(); plt.show()
except Exception as e:
    print("[scatter] skipped:", e)

try:
    sil = silhouette_samples(X, pdf["cluster"].values)
    print("Overall silhouette:", round(float(sil.mean()), 4))
    tmp = pd.DataFrame({"cluster": pdf["cluster"].values, "silhouette": sil})
    ax = tmp.boxplot(by="cluster", column="silhouette", grid=False, figsize=(6,4))
    plt.suptitle(""); plt.title("Silhouette by cluster"); plt.xlabel("cluster"); plt.ylabel("silhouette")
    plt.show()
except Exception as e:
    print("[silhouette] skipped:", e)



In [0]:
pdf.groupby("cluster")[["pvp_propostas","lead_to_deal_days"]].median().plot.bar(figsize=(8,4))


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

sizes = pdf["cluster"].value_counts(normalize=True).mul(100).round(2)
sizes_df = sizes.reset_index().rename(columns={"index":"cluster", "cluster":"pct"})
plt.figure(figsize=(8,4))
sns.barplot(data=sizes_df, x="cluster", y="pct", palette="viridis")
plt.title("Cluster Distribution (%)")
plt.ylabel("Share of Total (%)")
plt.xlabel("Cluster")
for i,v in enumerate(sizes_df["pct"]):
    plt.text(i, v+0.5, f"{v:.1f}%", ha='center')
plt.tight_layout()
plt.show()


In [0]:
num_cols = ["pvp_propostas", "lead_to_deal_days", "overall_sales_duration_deals", "proposals_per_vehicle"]
cluster_summary = pdf.groupby("cluster")[num_cols].median().round(1)
cluster_summary["count"] = pdf["cluster"].value_counts().sort_index()
cluster_summary["%"] = (cluster_summary["count"] / cluster_summary["count"].sum() * 100).round(1)
display(cluster_summary)


In [0]:
from sklearn.preprocessing import StandardScaler

# Compute standardized medians
med = pdf.groupby("cluster")[num_cols].median()
med_std = pd.DataFrame(StandardScaler().fit_transform(med), 
                       index=med.index, columns=med.columns)

plt.figure(figsize=(10,5))
sns.heatmap(med_std, annot=True, cmap="Spectral", center=0, fmt=".2f")
plt.title("Cluster Profiles (Z-Score of Medians)")
plt.xlabel("Variables")
plt.ylabel("Cluster")
plt.tight_layout()
plt.show()


In [0]:
top_lead = (
    pdf.groupby(["cluster","lead_source"])
       .size()
       .groupby(level=0, group_keys=False)
       .apply(lambda x: (x / x.sum() * 100).nlargest(5))
       .reset_index(name="pct")
)

plt.figure(figsize=(10,6))
sns.barplot(data=top_lead, x="pct", y="lead_source", hue="cluster", palette="tab10")
plt.title("Top 5 Lead Sources per Cluster (%)")
plt.xlabel("Share within Cluster (%)")
plt.ylabel("Lead Source")
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()


In [0]:
from sklearn.metrics import silhouette_samples
import numpy as np

sil_vals = silhouette_samples(X, pdf["cluster"])
pdf["silhouette"] = sil_vals

plt.figure(figsize=(8,5))
sns.boxplot(data=pdf, x="cluster", y="silhouette", palette="coolwarm")
plt.title(f"Silhouette by Cluster (overall mean = {np.mean(sil_vals):.3f})")
plt.ylabel("Silhouette Coefficient")
plt.xlabel("Cluster")
plt.tight_layout()
plt.show()


In [0]:
svd2 = TruncatedSVD(n_components=2, random_state=42)
X2 = svd2.fit_transform(Xs)

plt.figure(figsize=(7,6))
sns.scatterplot(x=X2[:,0], y=X2[:,1], hue=pdf["cluster"], palette="Set2", alpha=0.4, s=20)
plt.title("2D Projection of Clusters (SVD components)")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()


In [0]:
# === IMPROVED CLUSTER PROFILING VISUALS ===
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# --- 1️⃣ Median numeric summary per cluster ---
num_cols = ["pvp_propostas", "lead_to_deal_days", "overall_sales_duration_deals", "proposals_per_vehicle"]
num_cols = [c for c in num_cols if c in pdf.columns]

cluster_summary = pdf.groupby("cluster")[num_cols].median().round(1)
cluster_summary["count"] = pdf["cluster"].value_counts().sort_index()
cluster_summary["%"] = (cluster_summary["count"] / cluster_summary["count"].sum() * 100).round(1)

print("=== Numeric medians per cluster ===")
display(cluster_summary)

# --- 2️⃣ Heatmap of normalized medians (z-score by column) ---
z = (cluster_summary[num_cols] - cluster_summary[num_cols].mean()) / cluster_summary[num_cols].std(ddof=0)
plt.figure(figsize=(10,5))
sns.heatmap(z, annot=True, cmap="RdYlGn", center=0, linewidths=0.5, fmt=".2f")
plt.title("Cluster Profiles (Z-Score of Numeric Medians)")
plt.ylabel("Cluster")
plt.show()

# --- 3️⃣ Relative differences heatmap (percent difference from mean) ---
rel = ((cluster_summary[num_cols] / cluster_summary[num_cols].mean() - 1) * 100).round(1)
plt.figure(figsize=(10,5))
sns.heatmap(rel, annot=True, cmap="coolwarm", center=0, linewidths=0.5, fmt=".1f")
plt.title("Cluster Profiles (% Difference from Mean)")
plt.ylabel("Cluster")
plt.show()

# --- 4️⃣ Distributions of key numeric variables ---
for c in num_cols:
    plt.figure(figsize=(6,3))
    sns.kdeplot(data=pdf, x=c, hue="cluster", common_norm=False, fill=True, alpha=0.3)
    plt.title(f"Distribution of {c} by Cluster")
    plt.show()

# --- 5️⃣ Improved 2D Projection with transparency ---
from sklearn.decomposition import TruncatedSVD

svd2 = TruncatedSVD(n_components=2, random_state=42)
X2 = svd2.fit_transform(Xs)

plt.figure(figsize=(7,6))
palette = sns.color_palette("tab10", n_colors=len(pdf["cluster"].unique()))
sns.scatterplot(
    x=X2[:,0], y=X2[:,1],
    hue=pdf["cluster"].astype(str),
    palette=palette, s=10, alpha=0.3, linewidth=0
)
plt.title("2D SVD Projection Colored by Cluster")
plt.xlabel("Component 1"); plt.ylabel("Component 2")
plt.legend(title="Cluster", ncol=2, fontsize=8)
plt.tight_layout()
plt.show()


In [0]:
# === Best-practice K-Means (no casts; pvp_propostas DROPPED for model, kept for profiling) ===
import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, silhouette_samples

warnings.filterwarnings("ignore", category=UserWarning, module="threadpoolctl")

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# ---------- 1) Load ----------
pdf = spark.table(SRC_TBL).toPandas()
pdf = pdf.loc[:, ~pdf.columns.duplicated()]  # safety
print("Rows:", len(pdf), "| Cols:", len(pdf.columns))
existing = set(pdf.columns)

# ---------- 2) Feature sets ----------
# pvp_propostas is NOT used for modeling (low variance) but we keep it for profiling
car_value_col = "pvp_propostas" if "pvp_propostas" in existing else None

# Strong numerics for the model
log_num_desired_model = ["lead_to_deal_days", "overall_sales_duration_deals"]  # log1p + robust scale
lin_num_desired_model = ["proposals_per_vehicle"]                               # robust scale

# Compact behavior categoricals
cat_cols_desired = [
    "lead_source", "deal_speed_category", "stage_deals",
    "tipo_cliente_contactos", "combustivel_propostas", "origem_do_negocio_deals"
]

# Drop irrelevant (modeling only)
manual_drop = [
    "concessao_contactos", "versao_propostas", "campaign_name_campaigns",
    "modelo_contactos", "proposta_realizada_stage_propostas", "motivo_da_perda_deals"
]

# Keep only existing
log_num = [c for c in log_num_desired_model if c in existing]
lin_num = [c for c in lin_num_desired_model if c in existing]
cat_cols = [c for c in cat_cols_desired if c in existing]
drop_cols = [c for c in manual_drop if c in existing]

Xdf = pdf.drop(columns=drop_cols)
print("Numerics (log) used in MODEL:", log_num)
print("Numerics (linear) used in MODEL:", lin_num)
print("Categoricals used in MODEL:", cat_cols)
if drop_cols: print("Dropped (modeling only):", drop_cols)
if car_value_col: print("Car value KEPT for profiling only:", car_value_col)
assert (log_num or lin_num or cat_cols), "No usable features."

# ---------- 3) Preprocess ----------
def _log1p(x): return np.log1p(x)

ohe_kw = {"handle_unknown": "ignore"}
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kw["sparse_output"] = True
else:
    ohe_kw["sparse"] = True
ohe_extra = {"min_frequency": 0.02, "max_categories": 25} if version.parse(sklearn.__version__) >= version.parse("1.1") else {}

stages = []
if log_num:
    stages.append((
        "num_log",
        Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("log", FunctionTransformer(_log1p, feature_names_out="one-to-one")),
            ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
        ]),
        log_num
    ))
if lin_num:
    stages.append((
        "num_lin",
        Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
        ]),
        lin_num
    ))
if cat_cols:
    stages.append((
        "cat",
        Pipeline([
            ("imp", SimpleImputer(strategy="constant", fill_value="__MISSING__")),
            ("ohe", OneHotEncoder(**ohe_kw, **ohe_extra)),
        ]),
        cat_cols
    ))

preprocess = ColumnTransformer(stages, remainder="drop", sparse_threshold=1.0)

# ---------- 4) Encode -> SVD (12 comps) -> whiten ----------
Xs = preprocess.fit_transform(Xdf)                        # sparse after OHE
n_comp = max(2, min(12, Xs.shape[1]-1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
expl = svd.explained_variance_ratio_.sum()
X = StandardScaler().fit_transform(X)                    # final whitening for KMeans
print(f"Sparse shape: {Xs.shape} | Reduced: {X.shape} | SVD explained variance: {expl:.3f}")

# ---------- 5) k-scan (Elbow + Silhouette) ----------
metrics = []
for k in range(2, 10):
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=30, max_iter=250, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_)))
    print(f"k={k} | silhouette={sil:.4f} | inertia={km.inertia_:.0f}")

m = pd.DataFrame(metrics, columns=["k","silhouette","inertia"])
best = m.sort_values(["silhouette","k"], ascending=[False, True]).iloc[0]
best_k, best_sil = int(best.k), float(best.silhouette)
print(f"\nBest k = {best_k} | silhouette = {best_sil:.4f}")

fig, ax = plt.subplots()
ax.plot(m.k, m.inertia, marker="o"); ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)")
ax.set_title("Elbow Curve (Inertia)"); plt.show()

fig, ax = plt.subplots()
ax.plot(m.k, m.silhouette, marker="o"); ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)")
ax.set_title("Silhouette vs k"); plt.show()

# ---------- 6) Final model + labels ----------
final = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=60, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = final.labels_

# ---------- 7) Profiling ----------
def quick_readout(df, cluster_col, value_col, buckets=(30, 60, 90), unit="d"):
    if value_col not in df.columns:
        return
    print("=== Quick readout per cluster ===")
    for k in sorted(df[cluster_col].dropna().unique()):
        g = df[df[cluster_col]==k][value_col].astype(float)
        if g.empty: continue
        med = np.nanmedian(g)
        parts = [(f"<= {b}{unit}", (g<=b).mean()*100) for b in buckets] + [(f"> {buckets[-1]}{unit}", (g>buckets[-1]).mean()*100)]
        print(f"Cluster {k}: median={med:.1f}{unit} | " + "  ".join([f"{a}={b:.1f}%" for a,b in parts]))

# Sizes
sizes = pdf["cluster"].value_counts(dropna=False).sort_index()
display(sizes.to_frame("count").assign(pct=(sizes/len(pdf)*100).round(2)))

# Behavioral KPI readout
if "lead_to_deal_days" in pdf.columns:
    quick_readout(pdf, "cluster", "lead_to_deal_days", buckets=(30,60,90), unit="d")

# Car value (profiling only)
if car_value_col:
    med_price = pdf.groupby("cluster")[car_value_col].median()
    print(f"\nMedian car value ({car_value_col}) by cluster:\n", med_price)

# Numeric heatmap (model numerics only)
model_num_cols = [c for c in (log_num + lin_num) if c in pdf.columns]
if model_num_cols:
    med = pdf.groupby("cluster")[model_num_cols].median(numeric_only=True)
    cols = [c for c in model_num_cols if c in med.columns]
    if cols:
        med_std = (med[cols] - med[cols].mean()) / med[cols].std(ddof=0)
        plt.figure(figsize=(min(12, 0.6*len(cols)+3), 0.5*len(med_std)+3))
        plt.imshow(med_std[cols], aspect="auto")
        plt.yticks(range(len(med_std.index)), med_std.index)
        plt.xticks(range(len(cols)), cols, rotation=60, ha="right")
        plt.title("Cluster z-score profiles (numeric medians) — MODEL FEATURES ONLY")
        plt.colorbar(); plt.tight_layout(); plt.show()

# 2-D projection & silhouette by cluster
try:
    svd2 = TruncatedSVD(n_components=2, random_state=SEED)
    X2 = svd2.fit_transform(Xs)
    plt.figure(figsize=(6,5))
    for k in sorted(pdf["cluster"].unique()):
        idx = (pdf["cluster"]==k).values
        plt.scatter(X2[idx,0], X2[idx,1], s=8, alpha=0.6, label=f"c{k}")
    plt.legend(title="cluster", ncol=2); plt.title("2-D projection (SVD) by cluster")
    plt.xlabel("Comp 1"); plt.ylabel("Comp 2"); plt.tight_layout(); plt.show()
except Exception as e:
    print("[scatter] skipped:", e)

try:
    sil = silhouette_samples(X, pdf["cluster"].values)
    print("Overall silhouette:", round(float(sil.mean()), 4))
    tmp = pd.DataFrame({"cluster": pdf["cluster"].values, "silhouette": sil})
    ax = tmp.boxplot(by="cluster", column="silhouette", grid=False, figsize=(6,4))
    plt.suptitle(""); plt.title("Silhouette by cluster"); plt.xlabel("cluster"); plt.ylabel("silhouette")
    plt.show()
except Exception as e:
    print("[silhouette] skipped:", e)


In [0]:
# === Improved K-Means (add key categoricals) ===
import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn
from packaging import version
from pyspark.sql import functions as F
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, silhouette_samples

warnings.filterwarnings("ignore", category=UserWarning, module="threadpoolctl")

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# ---------- 1) Load data ----------
sdf = spark.table(SRC_TBL)
pdf = sdf.toPandas()
pdf = pdf.loc[:, ~pdf.columns.duplicated()]
print("Rows:", len(pdf), "| Cols:", len(pdf.columns))
existing = set(pdf.columns)

# ---------- 2) Feature sets ----------
# Numeric variables (log + linear)
log_num = ["lead_to_deal_days", "overall_sales_duration_deals"]
lin_num = ["proposals_per_vehicle"]

# Categorical variables to add
cat_cols = ["lead_source", "combustivel_propostas", "deal_speed_category"]

# Ensure columns exist
log_num = [c for c in log_num if c in existing]
lin_num = [c for c in lin_num if c in existing]
cat_cols = [c for c in cat_cols if c in existing]
print("Numerics log-scaled:", log_num)
print("Numerics linear:", lin_num)
print("Categoricals:", cat_cols)
assert (log_num or lin_num or cat_cols), "No usable features."

# ---------- 3) Preprocessing ----------
def _log1p(x): return np.log1p(x)

ohe_kw = {"handle_unknown": "ignore"}
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kw["sparse_output"] = True
else:
    ohe_kw["sparse"] = True
ohe_extra = {"min_frequency": 0.02, "max_categories": 25} if version.parse(sklearn.__version__) >= version.parse("1.1") else {}

stages = []
if log_num:
    stages.append((
        "num_log",
        Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("log", FunctionTransformer(_log1p, feature_names_out="one-to-one")),
            ("sc", RobustScaler(with_centering=True, with_scaling=True)),
        ]),
        log_num
    ))
if lin_num:
    stages.append((
        "num_lin",
        Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc", RobustScaler(with_centering=True, with_scaling=True)),
        ]),
        lin_num
    ))
if cat_cols:
    stages.append((
        "cat",
        Pipeline([
            ("imp", SimpleImputer(strategy="constant", fill_value="__MISSING__")),
            ("ohe", OneHotEncoder(**ohe_kw, **ohe_extra)),
        ]),
        cat_cols
    ))

preprocess = ColumnTransformer(stages, remainder="drop", sparse_threshold=1.0)

# ---------- 4) Encode -> SVD -> whiten ----------
Xs = preprocess.fit_transform(pdf)
n_comp = max(2, min(12, Xs.shape[1]-1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
expl = svd.explained_variance_ratio_.sum()
X = StandardScaler().fit_transform(X)
print(f"Sparse shape: {Xs.shape} | Reduced: {X.shape} | SVD explained variance: {expl:.3f}")

# ---------- 5) k-scan (Elbow + Silhouette) ----------
metrics = []
for k in range(2, 10):
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=30, max_iter=250, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_)))
    print(f"k={k} | silhouette={sil:.4f} | inertia={km.inertia_:.0f}")

m = pd.DataFrame(metrics, columns=["k","silhouette","inertia"])
best = m.sort_values(["silhouette","k"], ascending=[False, True]).iloc[0]
best_k, best_sil = int(best.k), float(best.silhouette)
print(f"\nBest k = {best_k} | silhouette = {best_sil:.4f}")

fig, ax = plt.subplots(2,1,figsize=(6,8))
ax[0].plot(m.k, m.inertia, marker="o")
ax[0].set_xlabel("k"); ax[0].set_ylabel("Inertia (lower is better)"); ax[0].set_title("Elbow Curve (Inertia)")
ax[1].plot(m.k, m.silhouette, marker="o")
ax[1].set_xlabel("k"); ax[1].set_ylabel("Silhouette (higher is better)"); ax[1].set_title("Silhouette vs k")
plt.tight_layout(); plt.show()

# ---------- 6) Final K-Means ----------
final = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=60, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = final.labels_

# ---------- Profiling ----------
def quick_readout(df, cluster_col, value_col, buckets=(30, 60, 90), unit="d"):
    print("=== Quick readout per cluster ===")
    for k in sorted(df[cluster_col].dropna().unique()):
        g = df[df[cluster_col] == k][value_col].astype(float)
        if g.empty:
            continue
        med = np.nanmedian(g)
        parts = [(f"<= {b}{unit}", (g <= b).mean() * 100) for b in buckets]
        parts.append((f"> {buckets[-1]}{unit}", (g > buckets[-1]).mean() * 100))
        # ✅ fixed line (no escaping needed)
        print(
            f"Cluster {k}: median={med:.1f}{unit} | "
            + "  ".join([f"{a}={b:.1f}%" for (a, b) in parts])
        )

# Heatmap for numeric medians
num_cols = [c for c in (log_num + lin_num) if c in pdf.columns]
if num_cols:
    med = pdf.groupby("cluster")[num_cols].median(numeric_only=True)
    med_std = (med - med.mean()) / med.std(ddof=0)
    plt.figure(figsize=(8, 5))
    plt.imshow(med_std, aspect="auto", cmap="coolwarm")
    plt.yticks(range(len(med_std.index)), med_std.index)
    plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha="right")
    plt.title("Cluster z-score profiles (numeric medians)")
    plt.colorbar()
    plt.tight_layout()
    plt.show()

# Top categorical breakdowns
def top_cats(df, cluster_col, cat_col, topn=5):
    out = []
    for k, g in df.groupby(cluster_col):
        s = g[cat_col].astype(str).replace("nan", "__MISSING__")
        vcs = s.value_counts(normalize=True).head(topn)
        for val, frac in vcs.items():
            out.append([cat_col, int(k), val, float(round(frac * 100, 2))])
    return pd.DataFrame(out, columns=["column", "cluster", "value", "pct"])

for c in cat_cols:
    display(top_cats(pdf, "cluster", c, topn=5))

# 2D SVD Projection
svd2 = TruncatedSVD(n_components=2, random_state=SEED)
X2 = svd2.fit_transform(Xs)
plt.figure(figsize=(6, 5))
for k in sorted(pdf["cluster"].unique()):
    idx = (pdf["cluster"] == k).values
    plt.scatter(X2[idx, 0], X2[idx, 1], s=8, alpha=0.5, label=f"c{k}")
plt.legend(title="cluster", ncol=2)
plt.title("2D Projection by Cluster")
plt.xlabel("Comp 1")
plt.ylabel("Comp 2")
plt.tight_layout()
plt.show()

# Silhouette per cluster
sil = silhouette_samples(X, pdf["cluster"].values)
print("Overall silhouette:", round(float(sil.mean()), 4))
tmp = pd.DataFrame({"cluster": pdf["cluster"].values, "silhouette": sil})
tmp.boxplot(by="cluster", column="silhouette", grid=False, figsize=(6, 4))
plt.suptitle("")
plt.title("Silhouette by cluster")
plt.xlabel("cluster")
plt.ylabel("silhouette")
plt.show()


In [0]:
# === K-Means with lead_source + deal_speed_category (combustivel_propostas DROPPED) ===
import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, silhouette_samples

warnings.filterwarnings("ignore", category=UserWarning, module="threadpoolctl")

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# ---------- 1) Load ----------
pdf = spark.table(SRC_TBL).toPandas()
pdf = pdf.loc[:, ~pdf.columns.duplicated()]
print("Rows:", len(pdf), "| Cols:", len(pdf.columns))
existing = set(pdf.columns)

# ---------- 2) Features ----------
# Numerics for behavior
log_num = [c for c in ["lead_to_deal_days", "overall_sales_duration_deals"] if c in existing]
lin_num = [c for c in ["proposals_per_vehicle"] if c in existing]

# Categoricals (DROP combustivel_propostas)
cat_cols = [c for c in ["lead_source", "deal_speed_category"] if c in existing]

print("Numerics log-scaled:", log_num)
print("Numerics linear:", lin_num)
print("Categoricals:", cat_cols)
assert (log_num or lin_num or cat_cols), "No usable features."

# ---------- 3) Preprocess ----------
def _log1p(x): return np.log1p(x)

ohe_kw = {"handle_unknown": "ignore"}
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kw["sparse_output"] = True
else:
    ohe_kw["sparse"] = True
ohe_extra = {"min_frequency": 0.02, "max_categories": 25} if version.parse(sklearn.__version__) >= version.parse("1.1") else {}

num_log_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("log", FunctionTransformer(_log1p, feature_names_out="one-to-one")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])

num_lin_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])

cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="constant", fill_value="__MISSING__")),
    ("ohe", OneHotEncoder(**ohe_kw, **ohe_extra)),
])

transformers = []
if log_num: transformers.append(("num_log", num_log_pipe, log_num))
if lin_num: transformers.append(("num_lin", num_lin_pipe, lin_num))
if cat_cols: transformers.append(("cat", cat_pipe, cat_cols))

# Optional: re-balance numeric vs categorical influence
# Increase/decrease 'cat' weight (e.g., 0.6) to prevent dominance
weights = {"num_log": 1.0, "num_lin": 1.0, "cat": 0.7}  # <- tweak if desired

preprocess = ColumnTransformer(
    transformers,
    remainder="drop",
    sparse_threshold=1.0,
    transformer_weights={k: v for k, v in weights.items() if any(t[0] == k for t in transformers)}
)

# ---------- 4) Encode -> SVD -> whiten ----------
Xs = preprocess.fit_transform(pdf)
n_comp = max(2, min(12, Xs.shape[1]-1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
expl = svd.explained_variance_ratio_.sum()
X = StandardScaler().fit_transform(X)
print(f"Sparse shape: {Xs.shape} | Reduced: {X.shape} | SVD explained variance: {expl:.3f}")

# ---------- 5) k-scan ----------
metrics = []
for k in range(2, 11):
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=30, max_iter=250, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_)))
    print(f"k={k} | silhouette={sil:.4f} | inertia={km.inertia_:.0f}")

m = pd.DataFrame(metrics, columns=["k","silhouette","inertia"])
best = m.sort_values(["silhouette","k"], ascending=[False, True]).iloc[0]
best_k, best_sil = int(best.k), float(best.silhouette)
print(f"\nBest k = {best_k} | silhouette = {best_sil:.4f}")

fig, ax = plt.subplots(2,1,figsize=(6,8))
ax[0].plot(m.k, m.inertia, marker="o"); ax[0].set_xlabel("k"); ax[0].set_ylabel("Inertia"); ax[0].set_title("Elbow Curve")
ax[1].plot(m.k, m.silhouette, marker="o"); ax[1].set_xlabel("k"); ax[1].set_ylabel("Silhouette"); ax[1].set_title("Silhouette vs k")
plt.tight_layout(); plt.show()

# ---------- 6) Final model ----------
final = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=60, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = final.labels_

# ---------- 7) Profiling ----------
def quick_readout(df, cluster_col, value_col, buckets=(30,60,90), unit="d"):
    print("=== Quick readout per cluster ===")
    for k in sorted(df[cluster_col].dropna().unique()):
        g = df[df[cluster_col]==k][value_col].astype(float)
        if g.empty: continue
        med = np.nanmedian(g)
        parts = [(f"<= {b}{unit}", (g<=b).mean()*100) for b in buckets] + [(f"> {buckets[-1]}{unit}", (g>buckets[-1]).mean()*100)]
        print(f"Cluster {k}: median={med:.1f}{unit} | " + "  ".join([f"{a}={b:.1f}%" for a,b in parts]))

sizes = pdf["cluster"].value_counts().sort_index()
display(sizes.to_frame("count").assign(pct=(sizes/len(pdf)*100).round(2)))

if "lead_to_deal_days" in pdf.columns:
    quick_readout(pdf, "cluster", "lead_to_deal_days")

# Numeric heatmap
num_cols = [c for c in (log_num + lin_num) if c in pdf.columns]
if num_cols:
    med = pdf.groupby("cluster")[num_cols].median(numeric_only=True)
    med_std = (med - med.mean()) / med.std(ddof=0)
    plt.figure(figsize=(8,5))
    plt.imshow(med_std, aspect="auto", cmap="coolwarm")
    plt.yticks(range(len(med_std.index)), med_std.index)
    plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha="right")
    plt.title("Cluster z-score profiles (numeric medians)")
    plt.colorbar(); plt.tight_layout(); plt.show()

# Top categorical breakdowns
def top_cats(df, cluster_col, cat_col, topn=5):
    out=[]
    for k,g in df.groupby(cluster_col):
        s=g[cat_col].astype(str).replace("nan","__MISSING__")
        vc=s.value_counts(normalize=True).head(topn)
        for val, frac in vc.items(): out.append([cat_col,int(k),val,float(round(frac*100,2))])
    return pd.DataFrame(out, columns=["column","cluster","value","pct"])

for c in cat_cols:
    display(top_cats(pdf, "cluster", c, topn=5))

# 2D projection
svd2 = TruncatedSVD(n_components=2, random_state=SEED)
X2 = svd2.fit_transform(Xs)
plt.figure(figsize=(6,5))
for k in sorted(pdf["cluster"].unique()):
    idx = (pdf["cluster"]==k).values
    plt.scatter(X2[idx,0], X2[idx,1], s=8, alpha=0.5, label=f"c{k}")
plt.legend(title="cluster", ncol=2); plt.title("2D Projection by Cluster")
plt.xlabel("Comp 1"); plt.ylabel("Comp 2"); plt.tight_layout(); plt.show()

# Silhouette per cluster
sil = silhouette_samples(X, pdf["cluster"].values)
print("Overall silhouette:", round(float(sil.mean()), 4))
tmp = pd.DataFrame({"cluster": pdf["cluster"].values, "silhouette": sil})
tmp.boxplot(by="cluster", column="silhouette", grid=False, figsize=(6,4))
plt.suptitle(""); plt.title("Silhouette by cluster"); plt.xlabel("cluster"); plt.ylabel("silhouette")
plt.show()


In [0]:
# === FINAL K-MEANS (combustivel_propostas DROPPED) + BETTER GRAPHS ===
import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, silhouette_samples

warnings.filterwarnings("ignore", category=UserWarning, module="threadpoolctl")

SEED   = 42
SRC_TBL = "sc_gold.Features_Table"

# ------------------ Load ------------------
pdf = spark.table(SRC_TBL).toPandas()
pdf = pdf.loc[:, ~pdf.columns.duplicated()]
print("Rows:", len(pdf), "| Cols:", len(pdf.columns))
existing = set(pdf.columns)

# ------------------ Features (combustivel_propostas is intentionally DROPPED) ------------------
# Numerics (behavior)
log_num = [c for c in ["lead_to_deal_days", "overall_sales_duration_deals"] if c in existing]
lin_num = [c for c in ["proposals_per_vehicle"] if c in existing]

# Categoricals to USE
cat_cols = [c for c in ["lead_source", "deal_speed_category"] if c in existing]

print("Numerics (log):   ", log_num)
print("Numerics (linear):", lin_num)
print("Categoricals:     ", cat_cols)
assert (log_num or lin_num or cat_cols), "No usable features."

# ------------------ Preprocess ------------------
def _log1p(x): return np.log1p(x)

ohe_kw = {"handle_unknown": "ignore"}
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kw["sparse_output"] = True
else:
    ohe_kw["sparse"] = True

# reduce cardinality noise for cats if sklearn>=1.1
ohe_extra = {"min_frequency": 0.02, "max_categories": 25} if version.parse(sklearn.__version__) >= version.parse("1.1") else {}

num_log_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("log", FunctionTransformer(_log1p, feature_names_out="one-to-one")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])

num_lin_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])

cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="constant", fill_value="__MISSING__")),
    ("ohe", OneHotEncoder(**ohe_kw, **ohe_extra)),
])

transformers = []
if log_num: transformers.append(("num_log", num_log_pipe, log_num))
if lin_num: transformers.append(("num_lin", num_lin_pipe, lin_num))
if cat_cols: transformers.append(("cat", cat_pipe, cat_cols))

# Slightly down-weight categoricals so numerics still drive shape
weights = {"num_log": 1.0, "num_lin": 1.0, "cat": 0.75}
preprocess = ColumnTransformer(
    transformers,
    remainder="drop",
    sparse_threshold=1.0,
    transformer_weights={k:v for k,v in weights.items() if any(t[0]==k for t in transformers)}
)

# ------------------ Encode -> SVD -> Whiten ------------------
Xs = preprocess.fit_transform(pdf)          # sparse after OHE
n_comp = max(2, min(8, Xs.shape[1]-1))      # 8 comps by default (tweak 6..12)
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
expl = svd.explained_variance_ratio_.sum()
X = StandardScaler().fit_transform(X)
print(f"Sparse: {Xs.shape} | Reduced: {X.shape} | SVD explained variance: {expl:.3f}")

# ------------------ k-scan ------------------
metrics = []
k_range = range(2, 13)
for k in k_range:
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=40, max_iter=300, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_)))
    print(f"k={k:2d} | silhouette={sil:.4f} | inertia={km.inertia_:.0f}")

m = pd.DataFrame(metrics, columns=["k","silhouette","inertia"])
best = m.sort_values(["silhouette","k"], ascending=[False, True]).iloc[0]
best_k, best_sil = int(best.k), float(best.silhouette)
print(f"\nBest k = {best_k} | silhouette = {best_sil:.4f}")

# -------- Plots: Elbow + Silhouette --------
fig, ax = plt.subplots(2,1,figsize=(6,8))
ax[0].plot(m.k, m.inertia, marker="o"); ax[0].set_title("Elbow Curve"); ax[0].set_xlabel("k"); ax[0].set_ylabel("Inertia")
ax[1].plot(m.k, m.silhouette, marker="o"); ax[1].set_title("Silhouette vs k"); ax[1].set_xlabel("k"); ax[1].set_ylabel("Silhouette")
plt.tight_layout(); plt.show()

# ------------------ Final model ------------------
final = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=60, max_iter=400, random_state=SEED).fit(X)
pdf["cluster"] = final.labels_

# ------------------ Better Profiling & Graphs ------------------
def quick_readout(df, cluster_col, value_col, buckets=(30,60,90), unit="d"):
    print("=== Quick readout per cluster ===")
    for k in sorted(df[cluster_col].dropna().unique()):
        g = df[df[cluster_col]==k][value_col].astype(float)
        if g.empty: continue
        med = np.nanmedian(g)
        parts = [(f"<= {b}{unit}", (g<=b).mean()*100) for b in buckets] + [(f"> {buckets[-1]}{unit}", (g>buckets[-1]).mean()*100)]
        print(f"Cluster {k}: median={med:.1f}{unit} | " + "  ".join([f"{a}={b:.1f}%" for a,b in parts]))

# 1) Cluster sizes (bar chart)
sizes = pdf["cluster"].value_counts().sort_index()
sizes_df = sizes.to_frame("count").assign(pct=(sizes/len(pdf)*100).round(2))
display(sizes_df)

plt.figure(figsize=(6,3))
plt.bar(sizes.index.astype(str), sizes.values)
plt.title("Cluster sizes"); plt.xlabel("cluster"); plt.ylabel("count"); plt.tight_layout(); plt.show()

# 2) KPI readout for lead_to_deal_days (edit if you want another KPI)
if "lead_to_deal_days" in pdf.columns:
    quick_readout(pdf, "cluster", "lead_to_deal_days", buckets=(30,60,90), unit="d")

# 3) Numeric medians -> z-score heatmap
num_cols = [c for c in (log_num + lin_num) if c in pdf.columns]
if num_cols:
    med = pdf.groupby("cluster")[num_cols].median(numeric_only=True)
    med_std = (med - med.mean()) / med.std(ddof=0)
    plt.figure(figsize=(8,4))
    plt.imshow(med_std, aspect="auto", cmap="coolwarm")
    plt.yticks(range(len(med_std.index)), med_std.index)
    plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha="right")
    plt.title("Cluster z-score profiles (numeric medians)")
    plt.colorbar(); plt.tight_layout(); plt.show()

# 4) Stacked bars for categoricals (proportions by cluster)
def stacked_bar(df, cat_col, cluster_col="cluster", topn=None, title=None):
    s = df[[cluster_col, cat_col]].copy()
    s[cat_col] = s[cat_col].astype("object").where(s[cat_col].notna(), "__MISSING__")
    # optional: keep topn categories (collapse rest)
    if topn:
        top_vals = s[cat_col].value_counts().nlargest(topn).index
        s.loc[~s[cat_col].isin(top_vals), cat_col] = "Other"
    tab = (s.groupby([cluster_col, cat_col]).size()
             .groupby(level=0).apply(lambda x: x/x.sum()*100)
             .unstack(fill_value=0)
             .sort_index())
    plt.figure(figsize=(max(7, 0.5*len(tab.columns)+4), 4))
    bottoms = np.zeros(len(tab))
    for c in tab.columns:
        plt.bar(tab.index.astype(str), tab[c].values, bottom=bottoms, label=c)
        bottoms += tab[c].values
    plt.legend(title=cat_col, bbox_to_anchor=(1.04,1), loc="upper left")
    plt.title(title or f"{cat_col}: % distribution by cluster")
    plt.xlabel("cluster"); plt.ylabel("%")
    plt.tight_layout(); plt.show()

for c in cat_cols:
    stacked_bar(pdf, c, topn=8, title=f"{c} — % within cluster")

# 5) 2-D SVD projection colored by cluster
svd2 = TruncatedSVD(n_components=2, random_state=SEED)
X2 = svd2.fit_transform(Xs)
plt.figure(figsize=(6,5))
for k in sorted(pdf["cluster"].unique()):
    idx = (pdf["cluster"]==k).values
    plt.scatter(X2[idx,0], X2[idx,1], s=8, alpha=0.6, label=f"c{k}")
plt.legend(title="cluster", ncol=2, bbox_to_anchor=(1.02, 1), loc="upper left")
plt.title("2D Projection by Cluster")
plt.xlabel("Comp 1"); plt.ylabel("Comp 2"); plt.tight_layout(); plt.show()

# 6) Silhouette by cluster (boxplot)
sil = silhouette_samples(X, pdf["cluster"].values)
print("Overall silhouette:", round(float(sil.mean()), 4))
tmp = pd.DataFrame({"cluster": pdf["cluster"].values, "silhouette": sil})
tmp.boxplot(by="cluster", column="silhouette", grid=False, figsize=(6,4))
plt.suptitle(""); plt.title("Silhouette by cluster"); plt.xlabel("cluster"); plt.ylabel("silhouette")
plt.tight_layout(); plt.show()


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

if "cluster" in pdf.columns:
    sizes = (pdf["cluster"]
             .value_counts(normalize=True)
             .mul(100)
             .rename("pct")
             .reset_index()
             .rename(columns={"index":"cluster"}))
    # sort clusters numerically if possible
    try:
        sizes["cluster"] = sizes["cluster"].astype(int)
        sizes = sizes.sort_values("cluster")
    except Exception:
        pass

    plt.figure(figsize=(8,4))
    ax = sns.barplot(data=sizes, x="cluster", y="pct", palette="viridis")
    ax.set_title("Cluster Distribution (%)")
    ax.set_ylabel("Share of Total (%)")
    ax.set_xlabel("Cluster")

    for p in ax.patches:
        ax.text(p.get_x()+p.get_width()/2, p.get_height()+0.5,
                f"{p.get_height():.1f}%", ha="center", va="bottom", fontsize=9)
    plt.tight_layout(); plt.show()
else:
    print("[info] 'cluster' column not found; skipping cluster distribution.")


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

need = {"cluster","lead_source"}
if need.issubset(pdf.columns):
    # within-cluster % and pick top 5 per cluster
    top_lead = (pdf.groupby(["cluster","lead_source"])
                  .size()
                  .groupby(level=0, group_keys=False)
                  .apply(lambda s: (s/s.sum()*100).nlargest(5))
                  .reset_index(name="pct"))

    # sort clusters numerically if possible
    try:
        top_lead["cluster"] = top_lead["cluster"].astype(int)
        top_lead = top_lead.sort_values(["cluster","pct"], ascending=[True, False])
    except Exception:
        pass

    plt.figure(figsize=(10,6))
    sns.barplot(data=top_lead, x="pct", y="lead_source", hue="cluster", palette="tab10")
    plt.title("Top 5 Lead Sources per Cluster (%)")
    plt.xlabel("Share within Cluster (%)")
    plt.ylabel("Lead Source")
    plt.legend(title="Cluster", bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout(); plt.show()
else:
    print(f"[info] Missing columns {need - set(pdf.columns)}; skipping top lead chart.")


In [0]:
from sklearn.metrics import silhouette_samples
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

if "cluster" in pdf.columns:
    # If you already computed X (reduced/whitened matrix) use that; else use Xs (encoded) or bail.
    try:
        sil_vals = silhouette_samples(X, pdf["cluster"])
    except NameError:
        sil_vals = silhouette_samples(Xs, pdf["cluster"])  # fallback if X not available

    pdf["silhouette"] = sil_vals

    plt.figure(figsize=(8,5))
    sns.boxplot(data=pdf, x="cluster", y="silhouette", palette="coolwarm")
    plt.title(f"Silhouette by Cluster (overall mean = {np.mean(sil_vals):.3f})")
    plt.ylabel("Silhouette Coefficient")
    plt.xlabel("Cluster")
    plt.tight_layout(); plt.show()
else:
    print("[info] 'cluster' column not found; skipping silhouette plot.")


In [0]:
from sklearn.decomposition import TruncatedSVD
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Use the sparse encoded matrix Xs if you have it; otherwise use X (already reduced) and pad if needed
try:
    svd2 = TruncatedSVD(n_components=2, random_state=42)
    X2 = svd2.fit_transform(Xs)     # preferred: start from encoded sparse matrix
except NameError:
    # if X exists but might be >2D or 1D
    X2 = X if X.shape[1] >= 2 else np.pad(X, ((0,0),(0,2-X.shape[1])), mode="constant")

plt.figure(figsize=(7,6))
sns.scatterplot(x=X2[:,0], y=X2[:,1], hue=pdf["cluster"], palette="Set2", alpha=0.45, s=18, linewidth=0)
plt.title("2D Projection of Clusters (SVD components)")
plt.xlabel("Component 1"); plt.ylabel("Component 2")
plt.legend(title="Cluster", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout(); plt.show()


In [0]:
# === FINAL K-MEANS (combustivel_propostas DROPPED) + METRICS (Silhouette, DBI, CH, Dunn) ===
import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import cdist

warnings.filterwarnings("ignore", category=UserWarning, module="threadpoolctl")

SEED   = 42
SRC_TBL = "sc_gold.Features_Table"

# ------------------ Load ------------------
pdf = spark.table(SRC_TBL).toPandas()
pdf = pdf.loc[:, ~pdf.columns.duplicated()]
print("Rows:", len(pdf), "| Cols:", len(pdf.columns))
existing = set(pdf.columns)

# ------------------ Features (combustivel_propostas is intentionally DROPPED) ------------------
# Numerics (behavior)
log_num = [c for c in ["lead_to_deal_days", "overall_sales_duration_deals"] if c in existing]
lin_num = [c for c in ["proposals_per_vehicle"] if c in existing]

# Categoricals to USE
cat_cols = [c for c in ["lead_source", "deal_speed_category"] if c in existing]

print("Numerics (log):   ", log_num)
print("Numerics (linear):", lin_num)
print("Categoricals:     ", cat_cols)
assert (log_num or lin_num or cat_cols), "No usable features."

# ------------------ Preprocess ------------------
def _log1p(x): return np.log1p(x)

ohe_kw = {"handle_unknown": "ignore"}
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kw["sparse_output"] = True
else:
    ohe_kw["sparse"] = True

# reduce cardinality noise for cats if sklearn>=1.1
ohe_extra = {"min_frequency": 0.02, "max_categories": 25} if version.parse(sklearn.__version__) >= version.parse("1.1") else {}

num_log_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("log", FunctionTransformer(_log1p, feature_names_out="one-to-one")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])

num_lin_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])

cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="constant", fill_value="__MISSING__")),
    ("ohe", OneHotEncoder(**ohe_kw, **ohe_extra)),
])

transformers = []
if log_num: transformers.append(("num_log", num_log_pipe, log_num))
if lin_num: transformers.append(("num_lin", num_lin_pipe, lin_num))
if cat_cols: transformers.append(("cat", cat_pipe, cat_cols))

# Slightly down-weight categoricals so numerics still drive shape
weights = {"num_log": 1.0, "num_lin": 1.0, "cat": 0.75}
preprocess = ColumnTransformer(
    transformers,
    remainder="drop",
    sparse_threshold=1.0,
    transformer_weights={k: v for k, v in weights.items() if any(t[0] == k for t in transformers)}
)

# ------------------ Encode -> SVD -> Whiten ------------------
Xs = preprocess.fit_transform(pdf)          # sparse after OHE
n_comp = max(2, min(8, Xs.shape[1]-1))      # 8 comps by default (tweak 6..12)
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
expl = svd.explained_variance_ratio_.sum()
X = StandardScaler().fit_transform(X)
print(f"Sparse: {Xs.shape} | Reduced: {X.shape} | SVD explained variance: {expl:.3f}")

# ------------------ k-scan ------------------
metrics = []
k_range = range(2, 13)
for k in k_range:
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=40, max_iter=300, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_)))
    print(f"k={k:2d} | silhouette={sil:.4f} | inertia={km.inertia_:.0f}")

m = pd.DataFrame(metrics, columns=["k","silhouette","inertia"])
best = m.sort_values(["silhouette","k"], ascending=[False, True]).iloc[0]
best_k, best_sil = int(best.k), float(best.silhouette)
print(f"\nBest k = {best_k} | silhouette = {best_sil:.4f}")

# -------- Plots: Elbow + Silhouette --------
fig, ax = plt.subplots(2,1,figsize=(6,8))
ax[0].plot(m.k, m.inertia, marker="o"); ax[0].set_title("Elbow Curve"); ax[0].set_xlabel("k"); ax[0].set_ylabel("Inertia")
ax[1].plot(m.k, m.silhouette, marker="o"); ax[1].set_title("Silhouette vs k"); ax[1].set_xlabel("k"); ax[1].set_ylabel("Silhouette")
plt.tight_layout(); plt.show()

# ------------------ Final model ------------------
final = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=60, max_iter=400, random_state=SEED).fit(X)
pdf["cluster"] = final.labels_

# ------------------ Validation metrics (DBI, CH, Dunn, Silhouette overall) ------------------
labels = pdf["cluster"].values

# Davies–Bouldin (lower is better)
dbi = davies_bouldin_score(X, labels)

# Calinski–Harabasz (higher is better)
ch = calinski_harabasz_score(X, labels)

# Dunn Index (higher is better)
def dunn_index(points: np.ndarray, labels: np.ndarray) -> float:
    points = np.asarray(points); labels = np.asarray(labels)
    uniq = np.unique(labels)

    # max intra-cluster diameter
    max_intra = 0.0
    for k in uniq:
        pts = points[labels == k]
        if len(pts) > 1:
            d = cdist(pts, pts, metric="euclidean")
            max_intra = max(max_intra, float(d.max()))

    # min inter-cluster separation
    min_inter = np.inf
    for i, ki in enumerate(uniq[:-1]):
        Xi = points[labels == ki]
        for kj in uniq[i+1:]:
            Xj = points[labels == kj]
            d = cdist(Xi, Xj, metric="euclidean")
            val = float(d.min())
            if val < min_inter:
                min_inter = val

    return float(min_inter / (max_intra + 1e-12))

dunn = dunn_index(X, labels)
sil_overall = silhouette_score(X, labels)

print("\n=== Cluster Quality Metrics ===")
print(f"Silhouette (overall)     : {sil_overall:.4f}  (higher is better)")
print(f"Davies–Bouldin Index     : {dbi:.4f}  (lower is better)")
print(f"Calinski–Harabasz Score  : {ch:.2f}  (higher is better)")
print(f"Dunn Index               : {dunn:.4f}  (higher is better)")


In [0]:
# === SELF-CONTAINED DBSCAN on sc_gold.Features_Table (matches your K-Means preprocessing) ===
import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cdist
from time import perf_counter

# quiet down threadpoolctl noise (Databricks quirk)
warnings.filterwarnings("ignore", category=UserWarning, module="threadpoolctl")

SEED   = 42
SRC_TBL = "sc_gold.Features_Table"

# ------------------ Load ------------------
pdf = spark.table(SRC_TBL).toPandas()
pdf = pdf.loc[:, ~pdf.columns.duplicated()]
existing = set(pdf.columns)
print("Rows:", len(pdf), "| Cols:", len(pdf.columns))

# ------------------ Features (same as your K-Means; combustivel_propostas intentionally DROPPED) ------------------
log_num = [c for c in ["lead_to_deal_days", "overall_sales_duration_deals"] if c in existing]
lin_num = [c for c in ["proposals_per_vehicle"] if c in existing]
cat_cols = [c for c in ["lead_source", "deal_speed_category"] if c in existing]
assert (log_num or lin_num or cat_cols), "No usable features."

print("Numerics (log):   ", log_num)
print("Numerics (linear):", lin_num)
print("Categoricals:     ", cat_cols)

# ------------------ Preprocess (IDENTICAL to your last cell) ------------------
def _log1p(x): return np.log1p(x)

ohe_kw = {"handle_unknown": "ignore"}
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kw["sparse_output"] = True
else:
    ohe_kw["sparse"] = True

ohe_extra = {"min_frequency": 0.02, "max_categories": 25} if version.parse(sklearn.__version__) >= version.parse("1.1") else {}

num_log_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("log", FunctionTransformer(_log1p, feature_names_out="one-to-one")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])

num_lin_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])

cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="constant", fill_value="__MISSING__")),
    ("ohe", OneHotEncoder(**ohe_kw, **ohe_extra)),
])

transformers = []
if log_num: transformers.append(("num_log", num_log_pipe, log_num))
if lin_num: transformers.append(("num_lin", num_lin_pipe, lin_num))
if cat_cols: transformers.append(("cat", cat_pipe, cat_cols))

weights = {"num_log": 1.0, "num_lin": 1.0, "cat": 0.75}  # same balance
preprocess = ColumnTransformer(
    transformers,
    remainder="drop",
    sparse_threshold=1.0,
    transformer_weights={k:v for k,v in weights.items() if any(t[0]==k for t in transformers)}
)

# ------------------ Encode -> SVD -> Whiten ------------------
Xs = preprocess.fit_transform(pdf)                 # sparse after OHE
n_comp = max(2, min(8, Xs.shape[1]-1))            # same default 8 comps
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
expl = svd.explained_variance_ratio_.sum()
X = StandardScaler().fit_transform(X)
print(f"Sparse: {Xs.shape} | Reduced: {X.shape} | SVD explained variance: {expl:.3f}")

# ------------------ FAST DBSCAN (single fit; eps from k-distance sample) ------------------
rng = np.random.default_rng(SEED)
n, d = X.shape
min_samples = max(2*d, 10)               # rule-of-thumb in low-dim space

sample_k = min(10000, n)                 # <=10k to avoid OOM
idx_s   = rng.choice(n, sample_k, replace=False) if n > sample_k else np.arange(n)
Xsamp   = X[idx_s]

t0 = perf_counter()
nn = NearestNeighbors(n_neighbors=min_samples, metric='euclidean')
nn.fit(Xsamp)
dists, _ = nn.kneighbors(Xsamp)
kth = np.sort(dists[:, -1])              # distance to k-th neighbor
eps = float(np.percentile(kth, 95))      # robust heuristic
print(f"[DBSCAN] min_samples={min_samples}, eps≈{eps:.4f} (95th pct k-distance on {len(Xsamp)} sample)")

t1 = perf_counter()
db = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
labels = db.fit_predict(X)
t2 = perf_counter()

pdf["cluster_dbscan"] = labels
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise    = int(np.sum(labels == -1))
print(f"[DBSCAN] clusters={n_clusters}, noise={n_noise} ({n_noise/n*100:.1f}%), fit_time={t2-t1:.1f}s, total={t2-t0:.1f}s")

# ------------------ Metrics on non-noise clusters ------------------
mask = labels != -1
if mask.sum() > 0 and len(np.unique(labels[mask])) > 1:
    sil = silhouette_score(X[mask], labels[mask])
    dbi = davies_bouldin_score(X[mask], labels[mask])
    chi = calinski_harabasz_score(X[mask], labels[mask])
    print(f"Silhouette (no-noise): {sil:.4f}  |  DBI: {dbi:.4f}  |  Calinski–Harabasz: {chi:.1f}")
else:
    sil = dbi = chi = None
    print("Not enough non-noise clusters for metrics.")

# ------------------ Quick visuals ------------------
# (a) k-distance curve with eps line
plt.figure(figsize=(6,3))
plt.plot(np.arange(len(kth)), kth)
plt.axhline(eps, color='r', ls='--', lw=1)
plt.title("k-distance curve (sample)")
plt.ylabel(f"dist to {min_samples}-NN"); plt.xlabel("sorted samples")
plt.tight_layout(); plt.show()

# (b) DBSCAN cluster sizes (including noise)
sizes = pd.Series(labels).value_counts().sort_index()
plt.figure(figsize=(6,3))
plt.bar(sizes.index.astype(str), sizes.values, color="tab:blue")
plt.title("DBSCAN cluster sizes (-1 = noise)")
plt.xlabel("label"); plt.ylabel("count")
plt.tight_layout(); plt.show()


In [0]:
%pip install scikit-learn-extra


In [0]:
# === SELF-CONTAINED PAM (CLARA-style) on sc_gold.Features_Table ===
# - Same features & preprocessing as your K-Means cell
# - Subsampled medoids (CLARA) + full assignment => fast on ~90k rows
# - Prints Silhouette / DBI / CH and shows two compact plots

import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, silhouette_samples
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist

# Quiet some noisy runtime warnings on DBR
warnings.filterwarnings("ignore", category=UserWarning, module="threadpoolctl")

# --- KMedoids from scikit-learn-extra ---
try:
    from sklearn_extra.cluster import KMedoids
except Exception as e:
    raise ImportError("Please run `%pip install scikit-learn-extra` once in a separate cell, then re-run this.") from e

SEED    = 42
RNG     = np.random.default_rng(SEED)
SRC_TBL = "sc_gold.Features_Table"

# ------------------ 1) Load ------------------
pdf = spark.table(SRC_TBL).toPandas()
pdf = pdf.loc[:, ~pdf.columns.duplicated()]
existing = set(pdf.columns)
print(f"Rows: {len(pdf)} | Cols: {len(pdf.columns)}")

# ------------------ 2) Features (same as K-Means; combustivel_propostas implicitly dropped) ------------------
log_num = [c for c in ["lead_to_deal_days", "overall_sales_duration_deals"] if c in existing]
lin_num = [c for c in ["proposals_per_vehicle"] if c in existing]
cat_cols = [c for c in ["lead_source", "deal_speed_category"] if c in existing]
assert (log_num or lin_num or cat_cols), "No usable features."

print("Numerics (log):   ", log_num)
print("Numerics (linear):", lin_num)
print("Categoricals:     ", cat_cols)

# ------------------ 3) Preprocess (identical shape to your K-Means) ------------------
def _log1p(x): return np.log1p(x)

ohe_kw = {"handle_unknown": "ignore"}
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kw["sparse_output"] = True
else:
    ohe_kw["sparse"] = True

ohe_extra = {"min_frequency": 0.02, "max_categories": 25} if version.parse(sklearn.__version__) >= version.parse("1.1") else {}

num_log_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("log", FunctionTransformer(_log1p, feature_names_out="one-to-one")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])
num_lin_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc",  RobustScaler(with_centering=True, with_scaling=True)),
])
cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="constant", fill_value="__MISSING__")),
    ("ohe", OneHotEncoder(**ohe_kw, **ohe_extra)),
])

transformers = []
if log_num: transformers.append(("num_log", num_log_pipe, log_num))
if lin_num: transformers.append(("num_lin", num_lin_pipe, lin_num))
if cat_cols: transformers.append(("cat", cat_pipe, cat_cols))

# keep cats slightly lighter so numerics guide geometry
weights = {"num_log": 1.0, "num_lin": 1.0, "cat": 0.75}
preprocess = ColumnTransformer(
    transformers,
    remainder="drop",
    sparse_threshold=1.0,
    transformer_weights={k:v for k,v in weights.items() if any(t[0]==k for t in transformers)}
)

Xs = preprocess.fit_transform(pdf)                 # sparse after OHE
n_comp = max(2, min(8, Xs.shape[1]-1))            # same default as before
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
X = StandardScaler().fit_transform(X)
print(f"Sparse: {Xs.shape} | Reduced: {X.shape} | SVD explained variance: {svd.explained_variance_ratio_.sum():.3f}")

# ------------------ 4) CLARA-style PAM ------------------
# Fast medoids on subsamples, then assign all points to nearest medoid
K_PAM        =  best_k if 'best_k' in globals() else 8   # set explicitly if you prefer
SAMPLE_SIZE  =  18000                                    # 12–20k is good for ~90k rows
N_SUBSAMPLES =  4                                        # 3–6 subsamples keeps it fast
METRIC       = "euclidean"

def assign_in_chunks(X_all, medoids, metric="euclidean", chunk=20000):
    """Nearest-medoid assignment in memory-safe chunks."""
    n = X_all.shape[0]
    out = np.empty(n, dtype=int)
    for i in range(0, n, chunk):
        j = min(i+chunk, n)
        D = pairwise_distances(X_all[i:j], medoids, metric=metric)
        out[i:j] = D.argmin(axis=1)
    return out

def clara_kmedoids(X_emb, k, sample_size, n_subsamples, metric="euclidean", rng=None):
    rng = np.random.default_rng() if rng is None else rng
    best_cost, best_medoids = np.inf, None
    for _ in range(n_subsamples):
        idx = rng.choice(X_emb.shape[0], size=min(sample_size, X_emb.shape[0]), replace=False)
        Xs = X_emb[idx]
        # medoids on subsample
        pam = KMedoids(n_clusters=k, metric=metric, method="alternate",
                       init="k-medoids++", max_iter=300, random_state=int(rng.integers(1e9)))
        pam.fit(Xs)
        med_sub = Xs[pam.medoid_indices_]
        # evaluate cost on ALL points cheaply
        D_full = pairwise_distances(X_emb, med_sub, metric=metric)
        cost = float(D_full.min(axis=1).sum())
        if cost < best_cost:
            best_cost, best_medoids = cost, med_sub
    # final assignment for all points
    labels_all = assign_in_chunks(X_emb, best_medoids, metric=metric, chunk=20000)
    return labels_all, best_medoids

pam_labels, pam_medoids = clara_kmedoids(X, K_PAM, SAMPLE_SIZE, N_SUBSAMPLES, metric=METRIC, rng=RNG)
pdf["pam_cluster"] = pam_labels

# ------------------ 5) Metrics ------------------
sil = silhouette_score(X, pam_labels) if K_PAM >= 2 else np.nan
dbi = davies_bouldin_score(X, pam_labels) if K_PAM >= 2 else np.nan
ch  = calinski_harabasz_score(X, pam_labels) if K_PAM >= 2 else np.nan
print(f"[PAM-CLARA] k={K_PAM} | Sil={sil:.4f} | DBI={dbi:.4f} | CH={ch:.1f}")

# ------------------ 6) Compact visuals ------------------
# (a) Cluster sizes
sizes = pd.Series(pam_labels).value_counts().sort_index()
plt.figure(figsize=(7,3))
plt.bar(sizes.index.astype(str), sizes.values)
plt.title("PAM (CLARA) cluster sizes"); plt.xlabel("cluster"); plt.ylabel("count"); plt.tight_layout(); plt.show()

# (b) Silhouette by cluster (boxplot)
sil_s = silhouette_samples(X, pam_labels)
tmp = pd.DataFrame({"cluster": pam_labels, "silhouette": sil_s})
ax = tmp.boxplot(by="cluster", column="silhouette", grid=False, figsize=(8,4))
plt.suptitle(""); plt.title(f"PAM (CLARA) — Silhouette by cluster (overall={sil:.3f})")
plt.xlabel("cluster"); plt.ylabel("silhouette"); plt.tight_layout(); plt.show()


In [0]:
# ========= DBSCAN — richer visuals =========
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples

# Safety: handle degenerate/noise-only cases
has_clusters = (labels != -1).sum() > 0 and np.unique(labels[labels!=-1]).size >= 1
has_2plus_clusters = (labels != -1).sum() > 0 and np.unique(labels[labels!=-1]).size >= 2

# Helper: 2D scatter with sampling for large n
def scatter_2d(X2, labels, title, max_points=40000, alpha=0.25, s=6):
    n = X2.shape[0]
    if n > max_points:
        rng = np.random.default_rng(42)
        idx = rng.choice(n, size=max_points, replace=False)
    else:
        idx = np.arange(n)

    lab_s = labels[idx]
    Xs = X2[idx]
    uniq = np.unique(lab_s)

    plt.figure(figsize=(6.5, 5))
    for u in uniq:
        m = lab_s == u
        # noise = -1 in gray
        if u == -1:
            plt.scatter(Xs[m, 0], Xs[m, 1], s=s, alpha=alpha, label="noise (-1)", c="tab:gray")
        else:
            plt.scatter(Xs[m, 0], Xs[m, 1], s=s, alpha=alpha, label=f"c{int(u)}")
    plt.legend(loc="best", fontsize=8, ncol=2, frameon=False)
    plt.title(title)
    plt.xlabel("SVD-1 (whitened)"); plt.ylabel("SVD-2 (whitened)")
    plt.tight_layout(); plt.show()

# (1) 2D projection of clusters (SVD-1 vs SVD-2)
scatter_2d(X[:, :2], labels, title=f"DBSCAN — 2D projection (eps≈{eps:.3f}, min_samples={min_samples})")

# (2) Core vs Border vs Noise breakdown
# A point is 'core' if DBSCAN.core_sample_indices_ contains it; 'border' otherwise (non-noise)
role = np.full(len(labels), "noise", dtype=object)
if hasattr(db, "core_sample_indices_"):
    core_idx = np.zeros(len(labels), dtype=bool)
    core_idx[db.core_sample_indices_] = True
    role[(labels!=-1) & core_idx] = "core"
    role[(labels!=-1) & ~core_idx] = "border"

counts = pd.Series(role).value_counts().reindex(["core","border","noise"]).fillna(0).astype(int)
plt.figure(figsize=(5.5,3))
plt.bar(counts.index.astype(str), counts.values)
plt.title("DBSCAN — point roles")
plt.xlabel("role"); plt.ylabel("count")
for i,v in enumerate(counts.values): plt.text(i, v, f"{v:,}", ha="center", va="bottom", fontsize=8)
plt.tight_layout(); plt.show()

# (3) Silhouette distribution (non-noise clusters only)
if has_2plus_clusters:
    mask = labels != -1
    sil_s = silhouette_samples(X[mask], labels[mask])
    tmp = pd.DataFrame({"cluster": labels[mask], "silhouette": sil_s})
    ax = tmp.boxplot(by="cluster", column="silhouette", grid=False, figsize=(8,4))
    plt.suptitle("")
    plt.title(f"DBSCAN — silhouette by cluster (overall={silhouette_score(X[mask], labels[mask]):.3f})")
    plt.xlabel("cluster"); plt.ylabel("silhouette")
    plt.tight_layout(); plt.show()
else:
    print("DBSCAN silhouette plot skipped (need ≥2 non-noise clusters).")

# (4) Distance-to-nearest core point (intuition for density)
# Uses kNN distances you already computed (kth). Show the eps line and a zoom-in.
plt.figure(figsize=(6.2,3.4))
plt.plot(np.arange(len(kth)), kth)
plt.axhline(eps, color="r", ls="--", lw=1, label=f"eps={eps:.3f}")
plt.title(f"k-distance (k={min_samples}) with eps line")
plt.xlabel("sorted samples"); plt.ylabel("distance")
plt.legend(frameon=False)
plt.tight_layout(); plt.show()

# (5) Per-cluster size bars with percentage
sizes = pd.Series(labels).value_counts().sort_index()
plt.figure(figsize=(6.5,3.2))
plt.bar(sizes.index.astype(str), sizes.values)
plt.title("DBSCAN — cluster sizes (-1 = noise)")
plt.xlabel("label"); plt.ylabel("count")
tot = sizes.sum()
for i,(lab,cnt) in enumerate(sizes.items()):
    pct = 100.0*cnt/tot
    plt.text(i, cnt, f"{cnt:,}\n({pct:.1f}%)", ha="center", va="bottom", fontsize=8)
plt.tight_layout(); plt.show()


In [0]:
# ========= PAM (CLARA) — richer visuals =========
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, pairwise_distances

K = len(np.unique(pam_labels))

# (1) 2D projection with medoids highlighted
def scatter_2d_with_medoids(X2, labels, medoids2, title, max_points=40000, alpha=0.25, s=6):
    n = X2.shape[0]
    if n > max_points:
        rng = np.random.default_rng(42)
        idx = rng.choice(n, size=max_points, replace=False)
    else:
        idx = np.arange(n)

    lab_s = labels[idx]
    Xs = X2[idx]
    uniq = np.unique(lab_s)

    plt.figure(figsize=(6.8,5.2))
    for u in uniq:
        m = lab_s == u
        plt.scatter(Xs[m, 0], Xs[m, 1], s=s, alpha=alpha, label=f"c{int(u)}")
    # overlay medoids
    plt.scatter(medoids2[:,0], medoids2[:,1], s=120, marker="X", edgecolors="k", linewidths=1.0, label="medoids")
    plt.legend(loc="best", fontsize=8, ncol=2, frameon=False)
    plt.title(title)
    plt.xlabel("SVD-1 (whitened)"); plt.ylabel("SVD-2 (whitened)")
    plt.tight_layout(); plt.show()

scatter_2d_with_medoids(X[:, :2], pam_labels, pam_medoids[:, :2],
                        title=f"PAM (CLARA) — 2D projection (k={K})")

# (2) Distance-to-own-medoid distribution per cluster (compact violin-like boxplots)
# Good to see cluster compactness and outliers
D = pairwise_distances(X, pam_medoids, metric="euclidean")
d_self = D[np.arange(D.shape[0]), pam_labels]
tmp = pd.DataFrame({"cluster": pam_labels, "dist_to_medoid": d_self})

ax = tmp.boxplot(by="cluster", column="dist_to_medoid", grid=False, figsize=(8,4))
plt.suptitle("")
plt.title("PAM — distance to own medoid by cluster")
plt.xlabel("cluster"); plt.ylabel("distance to medoid")
plt.tight_layout(); plt.show()

# (3) Silhouette by cluster (denser summary) — complements your boxplot
sil_s = silhouette_samples(X, pam_labels) if K >= 2 else None
if sil_s is not None:
    tmp2 = pd.DataFrame({"cluster": pam_labels, "silhouette": sil_s})
    means = tmp2.groupby("cluster")["silhouette"].mean().reindex(range(K), fill_value=np.nan)
    plt.figure(figsize=(6.4,3.0))
    plt.bar(np.arange(K).astype(str), means.values)
    plt.title(f"PAM — mean silhouette by cluster (overall={sil_s.mean():.3f})")
    plt.xlabel("cluster"); plt.ylabel("mean silhouette")
    for i,v in enumerate(means.values):
        if np.isfinite(v): plt.text(i, v, f"{v:.2f}", ha="center", va="bottom", fontsize=8)
    plt.tight_layout(); plt.show()
else:
    print("PAM silhouette bar skipped (need k ≥ 2).")

# (4) Cluster sizes with percentages (k already shown, but with % it’s handy)
sizes = pd.Series(pam_labels).value_counts().sort_index()
plt.figure(figsize=(6.5,3.2))
plt.bar(sizes.index.astype(str), sizes.values)
plt.title("PAM — cluster sizes")
plt.xlabel("cluster"); plt.ylabel("count")
tot = sizes.sum()
for i,(lab,cnt) in enumerate(sizes.items()):
    pct = 100.0*cnt/tot
    plt.text(i, cnt, f"{cnt:,}\n({pct:.1f}%)", ha="center", va="bottom", fontsize=8)
plt.tight_layout(); plt.show()


In [0]:
# ========= PAM (CLARA): cluster vs lead & speed =========
import numpy as np, pandas as pd, matplotlib.pyplot as plt

PAM_COL   = "pam_cluster"
LEAD_CAT  = "lead_source"
SPEED_CAT = "deal_speed_category"
LEAD_NUM  = "lead_to_deal_days"
TOP_K     = 10

assert PAM_COL in pdf.columns, f"Missing {PAM_COL} — run the PAM cell first."

def _exists(col): return col in pdf.columns

def _stacked_pct_bar(df, cluster_col, cat_col, top_k=10, title=""):
    vc_overall = df[cat_col].fillna("__MISSING__").value_counts()
    keep = set(vc_overall.head(top_k).index)
    cat_clean = df[cat_col].fillna("__MISSING__").where(df[cat_col].fillna("__MISSING__").isin(keep), other="Other")
    tmp = (pd.crosstab(df[cluster_col], cat_clean, normalize="index") * 100.0).sort_index()
    tmp = tmp[sorted(tmp.columns, key=lambda c: (c=="Other", -tmp[c].sum()))]
    ax = tmp.plot(kind="bar", stacked=True, figsize=(9,4), width=0.85, legend=True)
    ax.set_title(title); ax.set_xlabel("cluster"); ax.set_ylabel("share (%)")
    ax.legend(bbox_to_anchor=(1.02,1), loc="upper left", title=cat_col)
    plt.tight_layout(); plt.show()

dfp = pdf.copy()

# (1) cluster × lead_source
if _exists(LEAD_CAT):
    _stacked_pct_bar(dfp, PAM_COL, LEAD_CAT,
                     top_k=TOP_K, title="PAM — cluster × lead_source (% share)")
else:
    print(f"⚠️ '{LEAD_CAT}' not found, skipped.")

# (2) cluster × deal_speed_category
if _exists(SPEED_CAT):
    _stacked_pct_bar(dfp, PAM_COL, SPEED_CAT,
                     top_k=min(8, TOP_K), title="PAM — cluster × deal_speed_category (% share)")
else:
    print(f"⚠️ '{SPEED_CAT}' not found, skipped.")

# (3) numeric: lead_to_deal_days by cluster (boxplot + medians)
if _exists(LEAD_NUM) and dfp[PAM_COL].nunique() >= 1:
    order = sorted(dfp[PAM_COL].unique())
    data = [dfp.loc[dfp[PAM_COL]==c, LEAD_NUM].dropna().values for c in order]
    plt.figure(figsize=(9,4))
    plt.boxplot(data, showfliers=False, labels=[str(c) for c in order])
    plt.title("PAM — lead_to_deal_days by cluster"); plt.xlabel("cluster"); plt.ylabel("lead_to_deal_days")
    plt.tight_layout(); plt.show()

    med = dfp.groupby(PAM_COL)[LEAD_NUM].median().sort_index()
    cnt = dfp.groupby(PAM_COL)[LEAD_NUM].count().reindex(med.index).fillna(0).astype(int)
    plt.figure(figsize=(8,3))
    plt.bar(med.index.astype(str), med.values)
    for i,(m,c) in enumerate(zip(med.values, cnt.values)):
        plt.text(i, m, f"n={c}", ha="center", va="bottom", fontsize=8)
    plt.title("PAM — median lead_to_deal_days by cluster")
    plt.xlabel("cluster"); plt.ylabel("median days")
    plt.tight_layout(); plt.show()
else:
    if not _exists(LEAD_NUM):
        print(f"ℹ️ '{LEAD_NUM}' not found; numeric lead plot skipped.")


In [0]:
# ========= DBSCAN: cluster vs lead & speed =========
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# config — tweak as you like
DBSCAN_COL = "cluster_dbscan"        # from your DBSCAN cell
LEAD_CAT   = "lead_source"           # categorical "lead"
SPEED_CAT  = "deal_speed_category"   # categorical "speed"
LEAD_NUM   = "lead_to_deal_days"     # numeric "lead speed" proxy (optional)
TOP_K      = 10                      # show top-K categories + "Other" in stacked bars
INCLUDE_NOISE = False                # True to include label -1 as a cluster

assert DBSCAN_COL in pdf.columns, f"Missing {DBSCAN_COL} — run the DBSCAN cell first."

def _exists(col): return col in pdf.columns

def _stacked_pct_bar(df, cluster_col, cat_col, top_k=10, title=""):
    # prepare: keep top-K categories overall, bucket the rest as "Other"
    vc_overall = df[cat_col].fillna("__MISSING__").value_counts()
    keep = set(vc_overall.head(top_k).index)
    cat_clean = df[cat_col].fillna("__MISSING__").where(df[cat_col].fillna("__MISSING__").isin(keep), other="Other")
    tmp = (pd.crosstab(df[cluster_col], cat_clean, normalize="index") * 100.0).sort_index()
    tmp = tmp[sorted(tmp.columns, key=lambda c: (c=="Other", -tmp[c].sum()))]  # "Other" last
    # plot
    ax = tmp.plot(kind="bar", stacked=True, figsize=(9,4), width=0.85, legend=True)
    ax.set_title(title); ax.set_xlabel("cluster"); ax.set_ylabel("share (%)")
    ax.legend(bbox_to_anchor=(1.02,1), loc="upper left", title=cat_col)
    plt.tight_layout(); plt.show()

# choose subset (optionally drop noise)
dfd = pdf.copy()
if not INCLUDE_NOISE:
    dfd = dfd[dfd[DBSCAN_COL] != -1]

# (1) stacked % bars: cluster × lead_source
if _exists(LEAD_CAT):
    _stacked_pct_bar(dfd, DBSCAN_COL, LEAD_CAT,
                     top_k=TOP_K, title="DBSCAN — cluster × lead_source (% share)")
else:
    print(f"⚠️ '{LEAD_CAT}' not found, skipped.")

# (2) stacked % bars: cluster × deal_speed_category
if _exists(SPEED_CAT):
    _stacked_pct_bar(dfd, DBSCAN_COL, SPEED_CAT,
                     top_k=min(8, TOP_K), title="DBSCAN — cluster × deal_speed_category (% share)")
else:
    print(f"⚠️ '{SPEED_CAT}' not found, skipped.")

# (3) numeric distributions: lead_to_deal_days by cluster (boxplot + medians)
if _exists(LEAD_NUM) and not dfd.empty and dfd[DBSCAN_COL].nunique() >= 1:
    # Boxplot
    order = sorted(dfd[DBSCAN_COL].unique())
    data = [dfd.loc[dfd[DBSCAN_COL]==c, LEAD_NUM].dropna().values for c in order]
    plt.figure(figsize=(9,4))
    plt.boxplot(data, showfliers=False, labels=[str(c) for c in order])
    plt.title("DBSCAN — lead_to_deal_days by cluster"); plt.xlabel("cluster"); plt.ylabel("lead_to_deal_days")
    plt.tight_layout(); plt.show()

    # Medians bar (with counts)
    med = dfd.groupby(DBSCAN_COL)[LEAD_NUM].median().sort_index()
    cnt = dfd.groupby(DBSCAN_COL)[LEAD_NUM].count().reindex(med.index).fillna(0).astype(int)
    plt.figure(figsize=(8,3))
    plt.bar(med.index.astype(str), med.values)
    for i,(m,c) in enumerate(zip(med.values, cnt.values)):
        plt.text(i, m, f"n={c}", ha="center", va="bottom", fontsize=8)
    plt.title("DBSCAN — median lead_to_deal_days by cluster")
    plt.xlabel("cluster"); plt.ylabel("median days")
    plt.tight_layout(); plt.show()
else:
    if not _exists(LEAD_NUM):
        print(f"ℹ️ '{LEAD_NUM}' not found; numeric lead plot skipped.")


In [0]:
# -----------------------------------------------------------
# DBSCAN clustering on sc_gold.features_table (safe + robust)
# -----------------------------------------------------------

from pyspark.sql import functions as F
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import DBSCAN

# === PARAMETERS ===
SOURCE_TABLE = "sc_gold.features_table"
RESULT_TABLE = "sc_gold.features_table_dbscan"   # output (won't alter the source)
EPS = 0.5
MIN_SAMPLES = 5
# ===================

# 1) Read source + helper id (does not modify source)
sdf = spark.read.table(SOURCE_TABLE).withColumn("__row_id", F.monotonically_increasing_id())
sdf_original = sdf

# 2) Drop columns only for modeling (temporary)
cols_to_drop = [
    "concessao_contactos",
    "campaign_name_campaigns",
    "motivo_da_perda_deals",
    "proposta_realizada_stage_propostas",
]
existing_to_drop = [c for c in cols_to_drop if c in sdf.columns]
sdf_model = sdf.drop(*existing_to_drop)

# 3) TEMP null handling in Spark (in-memory only)
#    - numeric -> median (cast to double to avoid Decimal issues)
#    - categorical -> "missing"
string_cols  = [c for c, t in sdf_model.dtypes if t == "string"]
numeric_cols = [c for c, t in sdf_model.dtypes
                if t in ("double","float","int","bigint","smallint","tinyint") or t.startswith("decimal")]

# Cast numerics to double so percentile_approx returns double
for c in numeric_cols:
    sdf_model = sdf_model.withColumn(c, F.col(c).cast("double"))

# Fill numeric with median
if numeric_cols:
    median_exprs = [F.percentile_approx(F.col(c), 0.5).alias(c) for c in numeric_cols]
    median_vals  = sdf_model.select(*median_exprs).first().asDict()
    medians      = {c: float(v) for c, v in median_vals.items() if v is not None}
    if medians:
        sdf_model = sdf_model.fillna(medians)

# Fill categoricals with sentinel
if string_cols:
    sdf_model = sdf_model.fillna({c: "missing" for c in string_cols})

# 4) Convert to pandas for scikit-learn
pdf = sdf_model.toPandas()
row_id = pdf["__row_id"].copy()
X_df = pdf.drop(columns=["__row_id"])

# 5) Build preprocessing (sklearn >= 1.4)
num_cols = X_df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_df.select_dtypes(exclude=[np.number]).columns.tolist()

transformers = []
if num_cols:
    # with_mean=False so it works if final matrix is sparse
    numeric_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=False))
    ])
    transformers.append(("num", numeric_pipe, num_cols))

if cat_cols:
    categorical_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
    ])
    transformers.append(("cat", categorical_pipe, cat_cols))

# If both lists are empty, raise a friendly error
if not transformers:
    raise ValueError("No usable features found after drops. Please keep at least one numeric or categorical column.")

preprocess = ColumnTransformer(transformers)

# 6) Transform and cluster
X = preprocess.fit_transform(X_df)
db = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES, metric="euclidean")
labels = db.fit_predict(X)

# 7) Merge labels back to Spark (non-destructive) and save to a new table
labels_pdf = pd.DataFrame({"__row_id": row_id, "dbscan_label": labels.astype(int)})
labels_sdf = spark.createDataFrame(labels_pdf)

result_sdf = (
    sdf_original.join(labels_sdf, on="__row_id", how="left")
                .drop("__row_id")
)

(result_sdf.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(RESULT_TABLE)
)

display(result_sdf)
print(f"✅ Output table: {RESULT_TABLE}")
print(f"ℹ️ Dropped for modeling: {existing_to_drop}")
print(f"DBSCAN parameters: eps={EPS}, min_samples={MIN_SAMPLES}")
print("🔒 Original table remains unchanged.")


In [0]:
display(
  spark.table("sc_gold.features_table_dbscan")
       .groupBy("dbscan_label", "lead_source")
       .count()
       .orderBy("dbscan_label")
)


In [0]:
# -----------------------------------------------
# Cluster Profiling for DBSCAN results (PySpark)
# -----------------------------------------------
from pyspark.sql import functions as F, Window

RESULT_TABLE = "sc_gold.features_table_dbscan"   # change if needed
TOP_K = 3                                        # top K categories per cat column

sdf = spark.table(RESULT_TABLE)

# --- 0) Quick sanity check
if "dbscan_label" not in sdf.columns:
    raise ValueError("dbscan_label column not found. Make sure you ran the DBSCAN cell first.")

# --- 1) Cluster sizes + noise rate
cluster_sizes = (
    sdf.groupBy("dbscan_label")
       .count()
       .orderBy("dbscan_label")
)

total_n = sdf.count()
noise_n = sdf.filter(F.col("dbscan_label") == -1).count()
noise_rate = (noise_n / total_n) if total_n > 0 else 0.0

print(f"Total rows: {total_n:,}")
print(f"Noise rows (-1): {noise_n:,}  |  Noise rate: {noise_rate:.2%}")
display(cluster_sizes)

# --- 2) Identify numeric vs categorical columns
# treat booleans as categorical for readability
num_types = {"double","float","int","bigint","smallint","tinyint","decimal"}
cat_types = {"string","boolean"}

numeric_cols = [c for c, t in sdf.dtypes if (t in num_types or t.startswith("decimal")) and c != "dbscan_label"]
categorical_cols = [c for c, t in sdf.dtypes if t in cat_types and c != "dbscan_label"]

# --- 3) Numeric summary per cluster
numeric_summary = None
if numeric_cols:
    agg_exprs = []
    for c in numeric_cols:
        agg_exprs.extend([
            F.count(F.col(c)).alias(f"{c}__count"),
            F.mean(F.col(c)).alias(f"{c}__mean"),
            F.stddev_samp(F.col(c)).alias(f"{c}__std"),
            F.min(F.col(c)).alias(f"{c}__min"),
            F.expr(f"percentile_approx({c}, 0.5)").alias(f"{c}__median"),
            F.max(F.col(c)).alias(f"{c}__max"),
        ])
    numeric_summary = sdf.groupBy("dbscan_label").agg(*agg_exprs).orderBy("dbscan_label")
    display(numeric_summary)
else:
    print("No numeric columns detected for summary.")

# --- 4) Top-K categories per cluster for each categorical column
def topk_modes_for_column(df, col_name, k=3):
    # counts per (cluster, value)
    counts = (df.groupBy("dbscan_label", F.col(col_name).alias("value"))
                .count())
    # total per cluster for percentages
    totals = counts.groupBy("dbscan_label").agg(F.sum("count").alias("cluster_total"))
    joined = counts.join(totals, on="dbscan_label", how="left")\
                   .withColumn("pct", F.col("count")/F.col("cluster_total"))
    # rank within each cluster by count desc
    w = Window.partitionBy("dbscan_label").orderBy(F.col("count").desc(), F.col("value").asc())
    ranked = joined.withColumn("rank", F.row_number().over(w))\
                   .filter(F.col("rank") <= k)\
                   .withColumn("column", F.lit(col_name))\
                   .select("dbscan_label", "column", "value", "count", F.round("pct", 4).alias("pct"))
    return ranked

cat_topk = None
if categorical_cols:
    frames = [topk_modes_for_column(sdf, c, TOP_K) for c in categorical_cols]
    # union all
    cat_topk = frames[0]
    for f in frames[1:]:
        cat_topk = cat_topk.unionByName(f)
    cat_topk = cat_topk.orderBy("dbscan_label", "column", F.desc("count"))
    display(cat_topk)
else:
    print("No categorical columns detected for top-K analysis.")

# --- 5) (Optional) Quick per-cluster breakdown for a few key fields
# Replace these with fields you care about most.
KEY_CATS = [c for c in ["lead_source", "modelo_contactos", "deal_speed_category"] if c in sdf.columns]
for c in KEY_CATS:
    print(f"\n=== Distribution of {c} by cluster ===")
    dist = (
        sdf.groupBy("dbscan_label", F.col(c).alias(c))
           .count()
           .withColumn("pct_in_cluster",
                       F.round(F.col("count") / F.sum("count").over(Window.partitionBy("dbscan_label")), 4))
           .orderBy("dbscan_label", F.desc("count"))
    )
    display(dist)


In [0]:
from pyspark.sql import functions as F

sdf = spark.read.table("sc_gold.features_table")
string_cols = [c for c, t in sdf.dtypes if t == "string"]

# count unique values per string column
cardinality = (
    sdf.select([
        F.countDistinct(F.col(c)).alias(c) for c in string_cols
    ])
)

display(cardinality)


In [0]:
# Show column names and data types
df_info = spark.read.table("sc_gold.features_table")
display(df_info.dtypes)


In [0]:
# -----------------------------------------------------------
# DBSCAN on sc_gold.features_table  (updated drops + robust)
# -----------------------------------------------------------

from pyspark.sql import functions as F
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import DBSCAN

# === PARAMETERS ===
SOURCE_TABLE = "sc_gold.features_table"
RESULT_TABLE  = "sc_gold.features_table_dbscan"   # output (source remains untouched)
EPS           = 0.5   # tune after elbow if needed
MIN_SAMPLES   = 5
# ===================

# 1) Read + helper id
sdf = spark.read.table(SOURCE_TABLE).withColumn("__row_id", F.monotonically_increasing_id())
sdf_original = sdf

# 2) Drop columns ONLY for modeling (non-destructive)
cols_to_drop = [
    "concessao_contactos",                # high-card
    "proposta_realizada_stage_propostas", # leakage
    "stage_deals",                        # leakage
    "motivo_da_perda_deals",              # leakage
    "campaign_name_campaigns",            # high-card
    "versao_propostas"                    # very high-card
]
existing_to_drop = [c for c in cols_to_drop if c in sdf.columns]
sdf_model = sdf.drop(*existing_to_drop)

# 3) TEMP null handling in Spark (in-memory only)
#    - numeric -> median (cast to double to avoid Decimal issues)
#    - categorical -> "missing"
string_cols  = [c for c, t in sdf_model.dtypes if t == "string"]
numeric_cols = [c for c, t in sdf_model.dtypes
                if t in ("double","float","int","bigint","smallint","tinyint") or t.startswith("decimal")]

# Cast numerics to double so percentile_approx returns double
for c in numeric_cols:
    sdf_model = sdf_model.withColumn(c, F.col(c).cast("double"))

# Fill numeric with median
if numeric_cols:
    median_exprs = [F.percentile_approx(F.col(c), 0.5).alias(c) for c in numeric_cols]
    median_vals  = sdf_model.select(*median_exprs).first().asDict()
    medians      = {c: float(v) for c, v in median_vals.items() if v is not None}
    if medians:
        sdf_model = sdf_model.fillna(medians)

# Fill categoricals with sentinel
if string_cols:
    sdf_model = sdf_model.fillna({c: "missing" for c in string_cols})

# 4) To pandas for scikit-learn
pdf   = sdf_model.toPandas()
row_id = pdf["__row_id"].copy()
X_df   = pdf.drop(columns=["__row_id"])

# 5) Preprocessing (sklearn ≥ 1.4)
num_cols = X_df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_df.select_dtypes(exclude=[np.number]).columns.tolist()

transformers = []
if num_cols:
    numeric_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler",  StandardScaler(with_mean=False))  # safe if final is sparse
    ])
    transformers.append(("num", numeric_pipe, num_cols))

if cat_cols:
    categorical_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe",     OneHotEncoder(handle_unknown="ignore", sparse_output=True))
    ])
    transformers.append(("cat", categorical_pipe, cat_cols))

if not transformers:
    raise ValueError("No usable features found after drops.")

preprocess = ColumnTransformer(transformers)

# 6) Transform + DBSCAN
X  = preprocess.fit_transform(X_df)
db = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES, metric="euclidean")
labels = db.fit_predict(X)

# 7) Merge back + save as NEW table (source untouched)
labels_pdf = pd.DataFrame({"__row_id": row_id, "dbscan_label": labels.astype(int)})
labels_sdf = spark.createDataFrame(labels_pdf)

result_sdf = (
    sdf_original.join(labels_sdf, on="__row_id", how="left")
                .drop("__row_id")
)

(result_sdf.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(RESULT_TABLE)
)

# Quick cluster counts
display(result_sdf.groupBy("dbscan_label").count().orderBy("dbscan_label"))
print(f"✅ Output table: {RESULT_TABLE}")
print(f"ℹ️ Dropped for modeling: {existing_to_drop}")
print(f"DBSCAN params: eps={EPS}, min_samples={MIN_SAMPLES}")
print("🔒 Original table remains unchanged.")


In [0]:
# ========================================
# Cluster profiling for tuned DBSCAN table
# ========================================
from pyspark.sql import functions as F, Window

RESULT_TABLE = "sc_gold.features_table_dbscan"
TOP_K = 3

sdf = spark.table(RESULT_TABLE)

# Cluster sizes + noise rate
cluster_sizes = sdf.groupBy("dbscan_label").count().orderBy("dbscan_label")
total_n = sdf.count()
noise_n  = sdf.filter(F.col("dbscan_label") == -1).count()
print(f"Total rows: {total_n:,}")
print(f"Noise rows (-1): {noise_n:,}  |  Noise rate: {noise_n/total_n:.2%}")
display(cluster_sizes)

# Identify types
num_types = {"double","float","int","bigint","smallint","tinyint","decimal"}
cat_types = {"string","boolean"}
numeric_cols = [c for c, t in sdf.dtypes if (t in num_types or t.startswith("decimal")) and c != "dbscan_label"]
categorical_cols = [c for c, t in sdf.dtypes if t in cat_types and c != "dbscan_label"]

# Numeric summary
if numeric_cols:
    agg_exprs = []
    for c in numeric_cols:
        agg_exprs += [
            F.count(F.col(c)).alias(f"{c}__count"),
            F.mean(F.col(c)).alias(f"{c}__mean"),
            F.stddev_samp(F.col(c)).alias(f"{c}__std"),
            F.min(F.col(c)).alias(f"{c}__min"),
            F.expr(f"percentile_approx({c}, 0.5)").alias(f"{c}__median"),
            F.max(F.col(c)).alias(f"{c}__max"),
        ]
    display(sdf.groupBy("dbscan_label").agg(*agg_exprs).orderBy("dbscan_label"))

# Top-K categories per cluster
def topk(df, col, k=3):
    counts = df.groupBy("dbscan_label", F.col(col).alias("value")).count()
    totals = counts.groupBy("dbscan_label").agg(F.sum("count").alias("tot"))
    joined = counts.join(totals, "dbscan_label").withColumn("pct", F.col("count")/F.col("tot"))
    w = Window.partitionBy("dbscan_label").orderBy(F.desc("count"), F.asc("value"))
    return (joined.withColumn("rank", F.row_number().over(w))
                  .filter(F.col("rank") <= k)
                  .withColumn("column", F.lit(col))
                  .select("dbscan_label","column","value","count",F.round("pct",4).alias("pct")))

if categorical_cols:
    frames = [topk(sdf, c, TOP_K) for c in categorical_cols]
    out = frames[0]
    for f in frames[1:]:
        out = out.unionByName(f)
    display(out.orderBy("dbscan_label","column",F.desc("count")))


In [0]:
from pyspark.sql import functions as F

df = spark.table("sc_gold.features_table_dbscan")

# Noise rate
total = df.count()
noise = df.filter(F.col("dbscan_label") == -1).count()
print(f"Noise rate: {noise/total:.2%} ({noise:,}/{total:,})")

# Cluster sizes
display(df.groupBy("dbscan_label").count().orderBy(F.desc("count")))


In [0]:
display(
  df.groupBy("dbscan_label", "lead_source")
    .count()
    .orderBy("dbscan_label", F.desc("count"))
)


In [0]:
# ============================================================
# FAST Grid search for DBSCAN eps — overwrites sc_gold.features_table_dbscan
# ============================================================
from pyspark.sql import functions as F
import numpy as np
import pandas as pd
from time import perf_counter

from sklearn.cluster import DBSCAN
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ---------- CONFIG ----------
SOURCE_TABLE = "sc_gold.features_table"
RESULT_TABLE = "sc_gold.features_table_dbscan"
MIN_SAMPLES = 5
EPS_GRID = [0.8, 1.0, 1.2, 1.5, 2.0]
SAMPLE_ROWS = 15000          # use subset for eps testing
COLS_TO_DROP = [
    "concessao_contactos",
    "proposta_realizada_stage_propostas",
    "stage_deals",
    "motivo_da_perda_deals",
    "campaign_name_campaigns",
    "versao_propostas",
    "modelo_contactos",
    "origem_do_negocio_deals",
    "combustivel_propostas",
    "deal_speed_category",
    "Season",
    "is_month_end_deal",
    "lead_year",
    "lead_month"
]


#| Column                  | Reason for drop                                                                                                                                                 |
#| ----------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------- |
#| **deal_speed_category** | It’s just a binned version of `lead_to_deal_days`. Keeping both adds noise and unnecessary dimensionality — DBSCAN “double counts” time-to-close information.   |
#| **Season**              | Has only 4 unique values (Winter, Spring, etc.) and is *highly correlated* with `lead_month` and `lead_year`. Removing it avoids redundant distance dimensions. |


# ---------------------------

# === 1. Load and prepare sample ===
sdf = spark.read.table(SOURCE_TABLE).withColumn("__row_id", F.monotonically_increasing_id())
sdf = sdf.drop(*[c for c in COLS_TO_DROP if c in sdf.columns])

# quick fill
string_cols  = [c for c, t in sdf.dtypes if t == "string"]
numeric_cols = [c for c, t in sdf.dtypes if t in ("double","float","int","bigint","smallint","tinyint") or t.startswith("decimal")]
for c in numeric_cols:
    sdf = sdf.withColumn(c, F.col(c).cast("double"))

if numeric_cols:
    med_exprs = [F.percentile_approx(F.col(c), 0.5).alias(c) for c in numeric_cols]
    med_vals  = sdf.select(*med_exprs).first().asDict()
    medians   = {c: float(v) for c, v in med_vals.items() if v is not None}
    sdf = sdf.fillna(medians)
if string_cols:
    sdf = sdf.fillna({c: "missing" for c in string_cols})

# take a sample
sdf_sample = sdf.sample(withReplacement=False, fraction=min(SAMPLE_ROWS / sdf.count(), 1.0), seed=42)
pdf = sdf_sample.toPandas()
row_id = pdf["__row_id"].copy()
X_df = pdf.drop(columns=["__row_id"])

# === 2. Preprocess ===
num_cols = X_df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_df.select_dtypes(exclude=[np.number]).columns.tolist()

transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler",  StandardScaler(with_mean=False))
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe",     OneHotEncoder(handle_unknown="ignore", sparse_output=True))
    ]), cat_cols))
preprocess = ColumnTransformer(transformers)
X = preprocess.fit_transform(X_df)
n_rows = X.shape[0]

# === 3. Try several eps ===
summary = []
labels_cache = {}

for eps in EPS_GRID:
    t0 = perf_counter()
    db = DBSCAN(eps=float(eps), min_samples=MIN_SAMPLES, metric="euclidean")
    labels = db.fit_predict(X)
    dt = perf_counter() - t0

    labels_np = np.asarray(labels)
    noise_n = int((labels_np == -1).sum())
    noise_rate = noise_n / n_rows
    n_clusters = len(set(labels_np)) - (1 if -1 in labels_np else 0)
    summary.append({
        "eps": float(eps),
        "noise_rate": round(noise_rate, 4),
        "n_clusters": int(n_clusters),
        "seconds": round(dt, 2)
    })
    labels_cache[float(eps)] = labels_np

summary_pdf = pd.DataFrame(summary).sort_values("eps")
display(summary_pdf)

# pick best eps by lowest noise deviation (~0.3 target)
summary_pdf["score"] = abs(summary_pdf["noise_rate"] - 0.3)
best_eps = summary_pdf.loc[summary_pdf["score"].idxmin(), "eps"]
print(f"✅ Suggested eps = {best_eps} (based on sample results)")

# === 4. Run DBSCAN once on full dataset with best eps ===
full_pdf = sdf.toPandas()
row_id_full = full_pdf["__row_id"].copy()
X_df_full = full_pdf.drop(columns=["__row_id"])
X_full = preprocess.transform(X_df_full)

db_best = DBSCAN(eps=float(best_eps), min_samples=MIN_SAMPLES, metric="euclidean")
labels_full = db_best.fit_predict(X_full)

# === 5. Save back to table ===
labels_pdf = pd.DataFrame({"__row_id": row_id_full, "dbscan_label": labels_full.astype(int)})
labels_sdf = spark.createDataFrame(labels_pdf)
result_sdf = sdf.join(labels_sdf, on="__row_id", how="left").drop("__row_id")

(result_sdf.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(RESULT_TABLE))

print(f"📁 Overwritten table: {RESULT_TABLE}")
print(f"DBSCAN params: eps={best_eps}, min_samples={MIN_SAMPLES}")


In [0]:
# -----------------------------------------------------------
# DBSCAN (eps=2.0) with memory-safe pipeline (SVD + float32)
# -----------------------------------------------------------
from pyspark.sql import functions as F
import numpy as np
import pandas as pd
import gc

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import DBSCAN

# ===== Params =====
SOURCE_TABLE = "sc_gold.features_table"
RESULT_TABLE = "sc_gold.features_table_dbscan"   # overwritten
EPS = 2.0
MIN_SAMPLES = 5
SVD_COMPONENTS = 50          # dimensionality reducer (strongly recommended)
MAX_ROWS = None              # e.g., 50000 to sample for testing; None = use all
# ==================

# Columns to drop only for modeling
COLS_TO_DROP = [
    "concessao_contactos","proposta_realizada_stage_propostas","stage_deals",
    "motivo_da_perda_deals","campaign_name_campaigns","versao_propostas",
    "modelo_contactos","origem_do_negocio_deals","combustivel_propostas",
    "deal_speed_category","Season","model_in_campaign","is_month_end_deal",
    "lead_year","lead_month",
]

# 1) Read + helper id
sdf = spark.read.table(SOURCE_TABLE).withColumn("__row_id", F.monotonically_increasing_id())
sdf_original = sdf

# 2) Drop modeling-only columns
sdf_model = sdf.drop(*[c for c in COLS_TO_DROP if c in sdf.columns])

# 3) TEMP null handling (Spark-side; not persisted)
string_cols  = [c for c, t in sdf_model.dtypes if t == "string"]
numeric_cols = [c for c, t in sdf_model.dtypes if t in ("double","float","int","bigint","smallint","tinyint") or t.startswith("decimal")]

for c in numeric_cols:
    sdf_model = sdf_model.withColumn(c, F.col(c).cast("double"))

if numeric_cols:
    med_exprs = [F.percentile_approx(F.col(c), 0.5).alias(c) for c in numeric_cols]
    med_vals  = sdf_model.select(*med_exprs).first().asDict()
    medians   = {c: float(v) for c, v in med_vals.items() if v is not None}
    if medians:
        sdf_model = sdf_model.fillna(medians)

if string_cols:
    sdf_model = sdf_model.fillna({c: "missing" for c in string_cols})

# 4) Optional: sample rows to keep memory in check during development
if MAX_ROWS:
    frac = min(MAX_ROWS / sdf_model.count(), 1.0)
    sdf_model = sdf_model.sample(False, frac, seed=42)

# 5) To pandas for scikit-learn
pdf = sdf_model.toPandas()
row_id = pdf["__row_id"].to_numpy()
X_df  = pdf.drop(columns=["__row_id"])
del pdf; gc.collect()

# 6) Preprocess -> OneHot (sparse) + SVD (dense low-dim) + float32
num_cols = X_df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_df.select_dtypes(exclude=[np.number]).columns.tolist()

transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler",  StandardScaler(with_mean=False))
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe",     OneHotEncoder(handle_unknown="ignore", sparse_output=True))
    ]), cat_cols))

preprocess = ColumnTransformer(transformers)
X_sparse = preprocess.fit_transform(X_df)
del X_df; gc.collect()

# Dimensionality reduction (recommended for memory + speed)
svd = TruncatedSVD(n_components=min(SVD_COMPONENTS, X_sparse.shape[1]-1), random_state=42)
X = svd.fit_transform(X_sparse).astype(np.float32, copy=False)
del X_sparse; gc.collect()

# 7) DBSCAN
db = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES, metric="euclidean", n_jobs=-1)
labels = db.fit_predict(X).astype(int)
del X; gc.collect()

# 8) Merge back + save
labels_pdf = pd.DataFrame({"__row_id": row_id, "dbscan_label": labels})
labels_sdf = spark.createDataFrame(labels_pdf)

result_sdf = (
    sdf_original.join(labels_sdf, on="__row_id", how="left")
                .drop("__row_id")
)

(result_sdf.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(RESULT_TABLE)
)

display(result_sdf.groupBy("dbscan_label").count().orderBy("dbscan_label"))
print(f"📁 Overwritten table: {RESULT_TABLE}")
print(f"DBSCAN params: eps={EPS}, min_samples={MIN_SAMPLES}, svd_components={SVD_COMPONENTS}, dtype=float32")
print("🔒 Original table remains unchanged.")


In [0]:
from pyspark.sql import functions as F

TABLE = "sc_gold.features_table_dbscan"

# load
sdf = spark.table(TABLE)

# handle null labels as noise
sdf = sdf.withColumn("dbscan_label",
                     F.when(F.col("dbscan_label").isNull(), F.lit(-1)).otherwise(F.col("dbscan_label")))

# basic info
n_total = sdf.count()
n_noise = sdf.filter(F.col("dbscan_label") == -1).count()
n_clusters = sdf.select("dbscan_label").distinct().count() - (1 if n_noise > 0 else 0)

print(f"Total rows: {n_total:,}")
print(f"Noise rows (-1): {n_noise:,}  ({n_noise/n_total:.2%})")
print(f"Clusters found: {n_clusters}")


In [0]:
numeric_cols = [c for c, t in sdf.dtypes if t in ("double","float","int","bigint","smallint","tinyint") or t.startswith("decimal")]

agg_exprs = [F.count("*").alias("n")]
for c in numeric_cols:
    agg_exprs += [
        F.mean(F.col(c)).alias(f"{c}_mean"),
        F.stddev(F.col(c)).alias(f"{c}_std"),
        F.min(F.col(c)).alias(f"{c}_min"),
        F.max(F.col(c)).alias(f"{c}_max"),
    ]

num_profile = (
    sdf.groupBy("dbscan_label")
       .agg(*agg_exprs)
       .orderBy("dbscan_label")
)
display(num_profile)


In [0]:
import matplotlib.pyplot as plt

# Convert to Pandas for plotting
cluster_sizes = (
    sdf.groupBy("dbscan_label")
       .count()
       .orderBy("dbscan_label")
       .toPandas()
)

plt.figure(figsize=(10,5))
plt.bar(cluster_sizes["dbscan_label"], cluster_sizes["count"], color="skyblue")
plt.xlabel("DBSCAN Label")
plt.ylabel("Row Count")
plt.title("Cluster Sizes (Including Noise)")
plt.show()


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

pdf = sdf.toPandas()
num_cols = [c for c in pdf.columns if pdf[c].dtype in ("float64", "int64") and c not in ["dbscan_label"]]
cluster_summary = pdf.groupby("dbscan_label")[num_cols].mean()

plt.figure(figsize=(12,8))
sns.heatmap(cluster_summary, cmap="viridis", annot=False)
plt.title("Cluster Mean Feature Heatmap")
plt.xlabel("Numeric Features")
plt.ylabel("DBSCAN Cluster")
plt.show()


In [0]:
from sklearn.decomposition import PCA
import numpy as np

pdf_sample = pdf.sample(n=min(30000, len(pdf)), random_state=42)
num_cols = [c for c in pdf_sample.columns if np.issubdtype(pdf_sample[c].dtype, np.number) and c != "dbscan_label"]

X = pdf_sample[num_cols].fillna(0)
pca = PCA(n_components=2, random_state=42)
coords = pca.fit_transform(X)

plt.figure(figsize=(10,8))
plt.scatter(coords[:,0], coords[:,1],
            c=pdf_sample["dbscan_label"],
            cmap="tab10",
            s=10, alpha=0.6)
plt.title("2D PCA Cluster Visualization (sampled)")
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.show()


In [0]:
display(
    sdf.filter(F.col("dbscan_label") == -1)
       .groupBy("lead_source")
       .count()
       .orderBy(F.desc("count"))
)


In [0]:
# all columns that were dropped for modeling
COLS_TO_DROP = [
    "concessao_contactos", "proposta_realizada_stage_propostas", "stage_deals",
    "motivo_da_perda_deals", "campaign_name_campaigns", "versao_propostas",
    "modelo_contactos", "origem_do_negocio_deals", "combustivel_propostas",
    "deal_speed_category", "Season", "model_in_campaign", "is_month_end_deal",
    "lead_year", "lead_month"
]

# read the DBSCAN table
sdf = spark.table("sc_gold.features_table_dbscan")

# keep only used features + dbscan label
used_features = [c for c in sdf.columns if c not in COLS_TO_DROP + ["__row_id"]]
sdf = sdf.select(used_features)


In [0]:
numeric_cols = [c for c, t in sdf.dtypes if t in ("double","float","int","bigint","smallint","tinyint") or t.startswith("decimal")]
string_cols  = [c for c, t in sdf.dtypes if t == "string"]

print("Numeric used:", numeric_cols)
print("Categorical used:", string_cols)


In [0]:
from pyspark.sql import functions as F

# Basic metrics
total = sdf.count()
noise = sdf.filter(F.col("dbscan_label") == -1).count()
clusters = sdf.select("dbscan_label").distinct().count() - (1 if noise > 0 else 0)

print(f"📊 Total rows: {total:,}")
print(f"🌀 Clusters found: {clusters}")
print(f"⚠️ Noise rows (-1): {noise:,} ({noise/total:.2%})")

# Cluster size table
display(
    sdf.groupBy("dbscan_label")
       .count()
       .orderBy("dbscan_label")
)


In [0]:
numeric_cols = [c for c, t in sdf.dtypes if t in ("double","float","int","bigint","smallint","tinyint") or t.startswith("decimal")]

agg_exprs = [F.count("*").alias("n")]
for c in numeric_cols:
    agg_exprs += [
        F.mean(F.col(c)).alias(f"{c}_mean"),
        F.stddev(F.col(c)).alias(f"{c}_std"),
        F.min(F.col(c)).alias(f"{c}_min"),
        F.max(F.col(c)).alias(f"{c}_max"),
    ]

num_profile = (
    sdf.groupBy("dbscan_label")
       .agg(*agg_exprs)
       .orderBy("dbscan_label")
)
display(num_profile)


In [0]:
from pyspark.sql import Window

string_cols = [c for c, t in sdf.dtypes if t == "string"]

for c in string_cols:
    print(f"\n📊 Top categories for '{c}' by cluster:")
    top = (
        sdf.groupBy("dbscan_label", c)
           .count()
           .withColumn("pct", F.col("count") / F.sum("count").over(Window.partitionBy("dbscan_label")))
           .orderBy("dbscan_label", F.desc("count"))
    )
    display(top)


In [0]:
import matplotlib.pyplot as plt

cluster_sizes = (
    sdf.groupBy("dbscan_label")
       .count()
       .orderBy("dbscan_label")
       .toPandas()
)

plt.figure(figsize=(10,5))
plt.bar(cluster_sizes["dbscan_label"], cluster_sizes["count"], color="cornflowerblue")
plt.xlabel("DBSCAN Label")
plt.ylabel("Count")
plt.title("Cluster Sizes (Including Noise)")
plt.show()


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

pdf = sdf.toPandas()
num_cols = [c for c in pdf.columns if pdf[c].dtype in ("float64","int64") and c not in ["dbscan_label"]]

cluster_summary = pdf.groupby("dbscan_label")[num_cols].mean()

plt.figure(figsize=(12,6))
sns.heatmap(cluster_summary, cmap="coolwarm", annot=False)
plt.title("Cluster Mean Feature Heatmap")
plt.xlabel("Numeric Features")
plt.ylabel("Cluster")
plt.show()


In [0]:
from sklearn.decomposition import PCA
import numpy as np

pdf_sample = pdf.sample(n=min(30000, len(pdf)), random_state=42)
num_cols = [c for c in pdf_sample.columns if np.issubdtype(pdf_sample[c].dtype, np.number) and c != "dbscan_label"]

X = pdf_sample[num_cols].fillna(0)
pca = PCA(n_components=2, random_state=42)
coords = pca.fit_transform(X)

plt.figure(figsize=(10,8))
plt.scatter(coords[:,0], coords[:,1],
            c=pdf_sample["dbscan_label"],
            cmap="tab10",
            s=10, alpha=0.6)
plt.title("2D PCA Projection of DBSCAN Clusters")
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.show()
